In [ ]:
"""
Project: Top-n Music Genre Classification Neural Network and CLI Application
File Name: Pipeline Collab - Elvin's local branch
Authors:
- Elvin Carmona
- Mark Jordan, github: markyjordan
- Michael Levins
Created: 01/23/2024, Updated: 02/06/20224

---

Description: This file contains pipeline and a function for extracting spectrograms,
MFCC and STFT. 

Copyright: © 2024 Elvin Carmona, Mark Jordan, Michael Levins
"""

Download GTZAN and copy to the /content/ directory.

In [ ]:
import os
import site
from datasets import load_dataset
import shutil

dataset = load_dataset("marsyas/gtzan", trust_remote_code=True)

# Path to the dataset is in the /root/.cache/
# This path appears to just contain the audio files only. There may be another
# directory along this path that contains spectrograms, but if not, we can make
# them ourselves!
dataset_cache_dir = './archive/Data/genres_original/'

# Destination directory in /content/
destination_dir = 'content/gtzan-dataset'

if os.path.exists(dataset_cache_dir):
    shutil.copytree(dataset_cache_dir, destination_dir)
    print("Dataset copied to:", destination_dir)
else:
    print("Dataset directory not found:", dataset_cache_dir)

Read in Audio Files:

In [3]:
from glob import glob

# Pattern to match .wav files in all genre directories
pattern = './training_data/gtzan-dataset/*/*.wav'

# Find all files matching the pattern
training_audio_files = glob(pattern)


# Print the number of audio files found
length = len(training_audio_files)
print(training_audio_files[0])
print("Number of .wav files in training dataset: ", length)




./training_data/gtzan-dataset/classical/classical.00038.wav
Number of .wav files in training dataset:  980


In [4]:
import numpy as np 
import librosa
import os

SAMPLE_RATE = 22050
MAP_MATCHING = {'classical': 0, 'blues': 1, 'country': 2, 
                'disco': 3, 'hiphop': 4, 'jazz': 5, 'metal': 6, 'pop': 7, 
                'reggae': 8, 'rock': 9}

def save_features_to_dict(audio_files):

    # dictinary to save data to return
    data_dict = {'label': [],
                'features':[]
                }
    indicator_left = "{"
    indicator_right = "}"
    arrow = ""
    line = "="
    head = "=>"
    # len(audio_files)
    for i in range(0, len(audio_files)): # gets test data
        # pulls out genre from audio files
        genre_label = (os.path.basename(audio_files[i]).split('.')[0])
        if (i % 50 == 0):
            arrow += line
        if (i % 10 == 0):
            percent = " " + str(round((i / len(audio_files))*100, 1)) + "%"
            print(indicator_left + arrow + head + indicator_right + percent, end='\r')
        y, sr = librosa.load(audio_files[i], sr=SAMPLE_RATE)
        # y = numpy array of audio data
        # sr = sample rate
            
        # add label 
        data_dict['label'].append(MAP_MATCHING[genre_label])
        
        # y = librosa.util.normalize(y, norm=np.inf, fill=False)        
    
        # Extract the envelope (min and max) and the mean of each feature along the 
        # time axis. This way, we will have a feature with constant size
        
        # # MFCC 
        mfcc = np.array(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20))
        mfcc_mean = mfcc.mean(axis=1)
        mfcc_min = mfcc.min(axis=1)
        mfcc_max = mfcc.max(axis=1)
        mfcc_feature = np.concatenate((mfcc_mean, mfcc_min, mfcc_max))
        
        # Compute the Chromagram
        chroma = np.array(librosa.feature.chroma_stft(y=y, sr=sr))
        chroma_mean = chroma.mean(axis=1)
        chroma_min = chroma.min(axis=1)
        chroma_max = chroma.max(axis=1)
        chroma_feature = np.concatenate((chroma_mean, chroma_min, chroma_max))
        
        # # # Compute Tonnetz
        tonnetz = np.array(librosa.feature.tonnetz(y=y, sr=sr))
        tntz_mean  = tonnetz.mean(axis=1)
        tntz_min =  tonnetz.min(axis=1)
        tntz_max = tonnetz.max(axis=1)
        tntz_feature = np.concatenate((tntz_mean, tntz_min, tntz_max))
        
        # # # Mel Spectograms
        mel_spec = np.array(librosa.feature.melspectrogram(y=y, sr=sr))
        ms_mean = mel_spec.mean(axis=1)
        ms_min = mel_spec.min(axis=1)
        ms_max = mel_spec.max(axis=1)
        mel_spec_feature = np.concatenate((ms_mean, ms_min, ms_max))
        
        ### working up to here ####  
        
        # STFT
        stft = np.array(np.abs(librosa.stft(y=y)))
        stft_mean = stft.mean(axis=1)
        stft_min = stft.min(axis=1)
        stft_max = stft.max(axis=1)
        stft_feature = np.concatenate((stft_mean, stft_min, stft_max))
           
        # zero_crossings = librosa.feature.zero_crossing_rate(y) 
        # zc_mean = zero_crossings.mean(axis=1)
        # zc_min = zero_crossings.min(axis=1)
        # zc_max = zero_crossings.max(axis=1)
        # zc_feature =  np.concatenate((zc_mean, zc_min, zc_max))      
        
        # # harmonic and percussive components
        D_harmonic, D_percussive = np.array(librosa.decompose.hpss(stft, margin=4))
        D_harmonic_mean = D_harmonic.mean(axis=1)
        D_harmonic_min  = D_harmonic.min(axis=1)
        D_harmonic_max  = D_harmonic.max(axis=1)
        D_harmonic_feature = np.concatenate((D_harmonic_mean, D_harmonic_min, D_harmonic_max))
        
        D_percussive_mean = D_percussive.mean(axis=1)
        D_percussive_min  = D_percussive.min(axis=1)
        D_percussive_max  = D_percussive.max(axis=1)
        D_percussive_feature = np.concatenate((D_percussive_mean, D_percussive_min, D_percussive_max))
        
        data_dict['features'].append(np.concatenate((chroma_feature, mel_spec_feature, mfcc_feature, 
                                                tntz_feature, stft_feature, D_harmonic_feature, D_percussive_feature)))
        
        
    return data_dict


if __name__ == "__main__":
    training_dict = save_features_to_dict(training_audio_files)
    print("Training files saved, length: ", len(training_dict['label']))
    


Training files saved, length:  980


In [5]:
print(training_dict['features'][1].shape)
# import tensorflow as tf
# from tensorflow import keras
# import numpy as np
# tf.config.experimental.list_physical_devices('GPU')
# print("TensorFlow version: ", tf.__version__)


(9783,)


<h2>CNN Starting point!</h2>

In [7]:
from keras.utils import to_categorical
# one hot encoding of labels
y_one_hot_train = to_categorical(training_dict['label'])

# shuffle arrays
permutations = np.random.permutation(980)

features = np.array(training_dict['features'])[permutations]
labels = np.array(y_one_hot_train)[permutations]

# split arrays into training, testing and validation
features_train = features[0:588]
labels_train = labels[0:588]

features_val = features[589:785]
labels_val = labels[589:785]

features_test = features[785:]
labels_test = labels[785:]

<h4>Build the network architecture</h4>

In [13]:

from keras.layers import  Dense, Input, Dropout
from keras.models import Model
from keras import optimizers, losses, metrics, regularizers



# create model (currently working!)
inputs = Input(shape=(9783,), name='feature')
x = Dense(4000, activation="relu", name='dense_1', kernel_regularizer=regularizers.L2(0.001))(inputs)
x = Dropout(0.5)(x)
x = Dense(2000, activation="relu", name= 'dense_2', kernel_regularizer=regularizers.L2(0.001))(x)
x = Dropout(0.3)(x)
x = Dense(500, activation="relu", name= 'dense_3', kernel_regularizer=regularizers.L2(0.001))(x)
x = Dropout(0.3)(x)
outputs = Dense(10, activation="softmax", name='predictions')(x)

top_n_model = Model(inputs=inputs, outputs=outputs)

# working model 
top_n_model.compile(
    optimizer=optimizers.AdamW(learning_rate=0.001),
    loss=losses.CategoricalCrossentropy(),
    metrics=[metrics.CategoricalAccuracy()]
)



top_n_model.summary()



Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature (InputLayer)        [(None, 9783)]            0         
                                                                 
 dense_1 (Dense)             (None, 4000)              39136000  
                                                                 
 dropout_6 (Dropout)         (None, 4000)              0         
                                                                 
 dense_2 (Dense)             (None, 2000)              8002000   
                                                                 
 dropout_7 (Dropout)         (None, 2000)              0         
                                                                 
 dense_3 (Dense)             (None, 500)               1000500   
                                                                 
 dropout_8 (Dropout)         (None, 500)               0   

In [14]:
top_n_model.fit(x=features_train.tolist(), y= labels_train.tolist(), 
                verbose=1, validation_data=(features_val.tolist(), labels_val.tolist()), epochs=1000)

Epoch 1/1000
19/19 [==============================] - 5s 218ms/step - loss: 202.6498 - categorical_accuracy: 0.1497 - val_loss: 33.4783 - val_categorical_accuracy: 0.1582
Epoch 2/1000
19/19 [==============================] - 1s 47ms/step - loss: 28.8252 - categorical_accuracy: 0.2279 - val_loss: 10.6143 - val_categorical_accuracy: 0.3316
Epoch 3/1000
19/19 [==============================] - 1s 47ms/step - loss: 12.4491 - categorical_accuracy: 0.2857 - val_loss: 8.4464 - val_categorical_accuracy: 0.2653
Epoch 4/1000
19/19 [==============================] - 1s 47ms/step - loss: 9.4269 - categorical_accuracy: 0.2670 - val_loss: 8.2222 - val_categorical_accuracy: 0.3418
Epoch 5/1000
19/19 [==============================] - 1s 46ms/step - loss: 8.5546 - categorical_accuracy: 0.2585 - val_loss: 8.1005 - val_categorical_accuracy: 0.3061
Epoch 6/1000
19/19 [==============================] - 1s 46ms/step - loss: 8.2803 - categorical_accuracy: 0.2228 - val_loss: 7.8399 - val_categorical_accuracy

In [15]:
score = top_n_model.evaluate(x=features_test.tolist(), y=labels_test.tolist(), verbose=1)
print("Accuracy: "+ str(score[1]*100)+'%')

7/7 [==============================] - 0s 7ms/step - loss: 2.1328 - categorical_accuracy: 0.4513
Accuracy: 45.12820541858673%


<h4>Save model</h4>

In [ ]:
top_n_model.save('top_n_model_75.keras')

Clear all data from GPU to free memory! 

In [ ]:
from numba import cuda
from keras import backend

device = cuda.get_current_device()
device.reset()

backend.clear_session()

<h3>Load model and predict</h3>

In [ ]:
from keras.models import load_model

def load_saved_model(model):
    top_n_model_loaded = load_model(model)
    top_n_model_loaded.summary()
    return top_n_model_loaded

if __name__ == "__main__":
    loaded_model = load_saved_model('top_n_model_75.keras')

In [ ]:
import numpy as np 
import librosa
import os

SAMPLE_RATE = 22050

def save_features_to_dict_single(audio_file):

    # dictinary to save data to return
    data_dict = {'label': [],
                'features':[]
                }
    
    # pulls out genre from audio files
    genre_label = (os.path.basename(audio_file).split('.')[0])
    y, sr = librosa.load(audio_file, sr=SAMPLE_RATE)
    # y = numpy array of audio data
    # sr = sample rate
        
    # add label 
    data_dict['label'].append(genre_label)
     

    # Extract the envelope (min and max) and the mean of each feature along the 
    # time axis. This way, we will have a feature with constant size
    
    # # MFCC 
    mfcc = np.array(librosa.feature.mfcc(y=y, sr=sr))
    mfcc_mean = mfcc.mean(axis=1)
    mfcc_min = mfcc.min(axis=1)
    mfcc_max = mfcc.max(axis=1)
    mfcc_feature = np.concatenate((mfcc_mean, mfcc_min, mfcc_max))
    
    # Compute the Chromagram
    chroma = np.array(librosa.feature.chroma_stft(y=y, sr=sr))
    chroma_mean = chroma.mean(axis=1)
    chroma_min = chroma.min(axis=1)
    chroma_max = chroma.max(axis=1)
    chroma_feature = np.concatenate((chroma_mean, chroma_min, chroma_max))
    
    # # # Compute Tonnetz
    tonnetz = np.array(librosa.feature.tonnetz(y=y, sr=sr))
    tntz_mean  = tonnetz.mean(axis=1)
    tntz_min =  tonnetz.min(axis=1)
    tntz_max = tonnetz.max(axis=1)
    tntz_feature = np.concatenate((tntz_mean, tntz_min, tntz_max))
    
    # # # Mel Spectograms
    mel_spec = np.array(librosa.feature.melspectrogram(y=y, sr=sr))
    ms_mean = mel_spec.mean(axis=1)
    ms_min = mel_spec.min(axis=1)
    ms_max = mel_spec.max(axis=1)
    mel_spec_feature = np.concatenate((ms_mean, ms_min, ms_max))
    
    ### working up to here ####  
    
    # STFT
    stft = np.array(np.abs(librosa.stft(y=y)))
    stft_mean = stft.mean(axis=1)
    stft_min = stft.min(axis=1)
    stft_max = stft.max(axis=1)
    stft_feature = np.concatenate((stft_mean, stft_min, stft_max))
        
    # zero_crossings = librosa.feature.zero_crossing_rate(y) 
    # zc_mean = zero_crossings.mean(axis=1)
    # zc_min = zero_crossings.min(axis=1)
    # zc_max = zero_crossings.max(axis=1)
    # zc_feature =  np.concatenate((zc_mean, zc_min, zc_max))      
    
    # # # harmonic and percussive components
    D_harmonic, D_percussive = np.array(librosa.decompose.hpss(stft, margin=4))
    D_harmonic_mean = D_harmonic.mean(axis=1)
    D_harmonic_min  = D_harmonic.min(axis=1)
    D_harmonic_max  = D_harmonic.max(axis=1)
    D_harmonic_feature = np.concatenate((D_harmonic_mean, D_harmonic_min, D_harmonic_max))
    
    D_percussive_mean = D_percussive.mean(axis=1)
    D_percussive_min  = D_percussive.min(axis=1)
    D_percussive_max  = D_percussive.max(axis=1)
    D_percussive_feature = np.concatenate((D_percussive_mean, D_percussive_min, D_percussive_max))
    
    data_dict['features'].append(np.concatenate((chroma_feature, mel_spec_feature, mfcc_feature, 
                                            tntz_feature, stft_feature, D_harmonic_feature, D_percussive_feature)))
    # os.system("rm " + audio_file) # will uncomment this after testing
    return data_dict


if __name__ == "__main__":
    pass
    

In [ ]:
from glob import glob
from numpy import reshape


def model_predict(feature, model):
    results = model.predict_on_batch(x=reshape(feature, (1, 9723))) # shape should be what the model is set to
    return results

if __name__ == "__main__":
    # Pattern to match .wav files in all genre directories
    
    pattern = './demos/*/*.wav'

    # Find all files matching the pattern
    training_audio_files = glob(pattern)
    print(len(training_audio_files))
    results_dict = {'file': [], 'results':[], 'expected_genre': []}
    for i in range(0, 1):
        features_dict = save_features_to_dict_single(training_audio_files[i])
        features = np.array(features_dict['features'])
        results_dict['expected_genre'].append(features_dict['label'])  
        results_dict['file'].append(training_audio_files[i])
        results_dict['results'].append(model_predict(features, loaded_model))


In [ ]:

def format_results(results_list):
    MAP_MATCHING = { 0: 'Classical', 1: 'Blues', 2:'Country', 
                3: 'Disco', 4: 'Hip Hop', 5: 'Jazz', 6: 'Metal', 7: 'Pop', 
                8: 'Reggae', 9: 'Rock'}
    results = [round(x*100, 2) for x in results_list]
    sorted_results = {}
    for i in range(0,len(results_list)):
        sorted_results[MAP_MATCHING[i]] = results[i]
    sorted_results = sorted(sorted_results.items(), key=lambda x:x[1], reverse=True)
    final_results_dict = dict(sorted_results)
    return final_results_dict


if __name__ == "__main__":
    pass



In [ ]:
def final_output_printer(formater_results_dict: dict, expected_genre:str):
    print("Expected genre: " + expected_genre)
    print("Model predictions in decending order:")
    counter = 1
    for keys in formater_results_dict:
        print("   " + str(counter) + ". " + keys + ": "+ str(formater_results_dict[keys]) + "%")
        counter += 1
if __name__ == "__main__":
    pass

In [ ]:
import os
import ffmpeg

def input_fortmater_mp4(file:str):
    file_renamed_mp3 = file.replace('.mp4', '.mp3')
    file_renamed_wav = file.replace('.mp4', '.wav')
    command2mp3 = "ffmpeg -i " + file + " " + file_renamed_mp3
    command2wav = "ffmpeg -i " + file_renamed_mp3 + " " +  file_renamed_wav
    os.system(command2mp3)
    os.system("rm " + file)
    os.system(command2wav)
    os.system("rm " + file_renamed_mp3)
    return file_renamed_wav


if __name__ == "__main__":
    pass  

In [ ]:
import os
import ffmpeg

def input_fortmater_mp3(file:str):
    file_renamed_wav = file.replace('.mp3', '.wav')
    command2mp3 = "ffmpeg -i " + file + " " + file_renamed_wav
    os.system(command2mp3)
    os.system("rm " + file)
    return file_renamed_wav

In [ ]:

from pytube import YouTube

def get_youtube_audio_file(file_url, genre):
    yt = YouTube(file_url)
    title = yt.title
    title = title.replace(" ", "")
    out_name = genre + '.' + title + '.mp4'
    audio = yt.streams.get_audio_only()
    audio_file = audio.download(filename=out_name)
    return audio_file
    
    
if __name__ == "__main__":
    pass

In [ ]:
# %pip install validators
import validators
import os

def input_checker_and_formatter(user_input:str, expected_genre:str):
    formatted_file = ""
    if user_input.endswith('.mp3'): # will need testing
        formatted_file = input_fortmater_mp3(user_input)
        os.rename(formatted_file, expected_genre + "." + formatted_file)
    elif user_input.endswith('.mp4'): # will need testing
        formatted_file = input_fortmater_mp4(user_input) 
        os.rename(formatted_file, expected_genre + "." + formatted_file)
    elif user_input.endswith('wav'): # will need testing
        print('File is ready for processing')
        os.rename(user_input, expected_genre + "." + user_input) 
    elif validators.url(user_input): # tested 
        retrived_file = get_youtube_audio_file(user_input, expected_genre)
        formatted_file = input_fortmater_mp4(os.path.basename(retrived_file))
    else:
        print('Error file type not accepted')
        return None # the calling function should interpret as an error and handle it
    return formatted_file
        
if __name__ == "__main__":
    pass 

In [ ]:
from numpy import argmax
MAP_MATCHING = { 0: 'classical', 1: 'blues', 2:'country', 
                3: 'disco', 4: 'hiphop', 5: 'jazz', 6: 'metal', 7: 'pop', 
                8: 'reggae', 9: 'rock'}

for i in range(0, len(results_dict['file'])):
    print("Test: " +str(i) + " Expected genre: " + (results_dict['expected_genre'][i][0]) + "\n      File:  " + (results_dict['file'][i])
          + "\nPredicted genre: " + (MAP_MATCHING[argmax(results_dict['results'][i])]))

In [ ]:
%pip install pytube
%pip install pydub
%pip install ffmpeg-python 
# if the ffmpeg does not work, do sudo apt install ffmpe in terminal (I had to do this because it didn't work)


In [ ]:
from pydub import AudioSegment
import os

# output file will be named 'genre + out_name'
# out_name must be have periods as separators if any
# such as sample.audio.file30s.wav and genre = 'genre.sample.audio.file30s.wav'
# will create two 30s .wav samples

def crop_file_to_30_sec(audio_file_path, out_name):
    t1 = 10000
    t2 = 40000
    t3 = 60000
    t4 = 90000
    waveFile = AudioSegment.from_file(audio_file_path)
    waveFile1 = waveFile[t1:t2]
    waveFile2 = waveFile[t3:t4]
    waveFile1.export(out_name + '.1.wav', format="wav")
    waveFile2.export(out_name + '.2.wav', format="wav")
    os.system("rm " + audio_file_path)
    

if __name__ == "__main__":
    pass

In [ ]:
## Expanding dataset - file names and url = 10/genre 
classical_dict = {'file_name': ['Tchaikovsky.PianoN1',
                                'Mozart.LNDF',
                                'Greig.MountKing',
                                'Elgar.MarchPomp',
                                'Schubert.MMN1',
                                'Rossini.BarberSevilla',
                                'Puccini.NessumDorma',
                                'Beethoven.PianokonsertN5',
                                'Bach.ToccataNFugue',
                                'HandelMessiah.hallelujah',
                                'FrederocChopin.FuneralMarch'
                                ],
                  'file_url' : ['https://youtu.be/P2l0lbn5TVg?si=2JOvTYoX2IFcTjOu',
                                'https://youtu.be/KPk90OeyZeg?si=IsM3NetmuYgzDieT',
                                'https://youtu.be/T9tTUayRFf4?si=-mGAYuVzPefSEEU1',
                                'https://youtu.be/CJCx_SCafTg?si=gh0dVpBlWcyujKf2',
                                'https://youtu.be/axb48YrvRmw?si=iDWgrNvcAZNiUCH9',
                                'https://youtu.be/pIf2zL6aCig?si=oDTswZmWy2B3gjH-',
                                'https://youtu.be/d_92G-Dgscw?si=YC5--OD-36p3MkoW',
                                'https://youtu.be/0IWRZprH8Zg?si=L0sSA3tFxG9W-5uD',
                                'https://youtu.be/IVJD3dL4diY?si=GEY47kizIpGhqZz-',
                                'https://youtu.be/usfiAsWR4qU?si=UIN9d5ZUxzQrFF-b',
                                'https://youtu.be/Hgw_RD_1_5I?si=TZXdQyinjjt2kKXO'
                               ]
                  }

blues_dict = {'file_name': ['GaryColeman.TheSkyisCrying',
                            'BuddyGuy.FleshNBone',
                            'AnnieLennoxIPutASpellOnYou',
                            'DavidDeacon.TheyAllSangAlong',
                            'WrecklessStrangers.FastGirls', 
                            'StevieRayVaughan.LittleWing', 
                            'Andra.AtLast',
                            'TheBarrBrothers.LordIJustCant',
                            'RichardThompson.IfLoveWhispersYour',
                            'TheBarrBrothers.HalfCrazy',
                            'DanielCastro.IllPlayTheBluesForYou'],
                  'file_url' : ['https://youtu.be/71Gt46aX9Z4?si=Z1yHamrUhtQ0asBi',
                                'https://youtu.be/-DXI4hxm_-o?si=Jwi60zuqz7a1povG',
                                'https://youtu.be/3TrSMaOZm3Y?si=GZKhPcMgjMSyRH3q',
                                'https://youtu.be/bGUHxMpKu8Y?si=c9rwxDEWSLNVJdA4',
                                'https://youtu.be/_a_opAYZd1I?si=EQ04bM8ah_XDUWZx', 
                                'https://youtu.be/Rj_NUS9hwxA?si=ZcpCqloxoXBAtIR2', 
                                'https://youtu.be/XzqY9ET9GYI?si=Gjxhk68qyApBWCEN',
                                'https://youtu.be/VgtYREp1mLQ?si=gB-daINFJbS7yvaA',
                                'https://youtu.be/gTmAseJHs-c?si=QTOTlb4EH0AwYdbW',
                                'https://youtu.be/MrpDVFbBmLY?si=hx1NaEf6RXtkZV4l',
                                'https://youtu.be/ioOzsi9aHQQ?si=xeFt8hjV1XM9SSxR']
                  }

country_dict = {'file_name': ['DariusRucker.WagonWheel',
                              'ChaseMatthew.WeAllGrowUp',
                              'LukeCombs.BeautifulCrazy',
                              'ShaniaTwain.Ifeellikeawoman',
                              'CodyJohnson.Thepainter', 
                              'KennyChesney.TakeHerHome',
                              'AlanJackson.RememberWhen',
                              'LadyAntebellum.NeedYouNow',
                              'Midlan.DrinkinProblem',
                              'BillliCurrington.PeopleareCrazy',
                              'LeeBrice.OneofThemGirls'],
                  'file_url' : ['https://youtu.be/hvKyBcCDOB4?si=snuAYT4BsmzZ6CFe',
                                'https://youtu.be/56sgEJPDy9g?si=nzBMB8hIxjYi4GrA',
                                'https://youtu.be/rItv9i6c7AY?si=g1GmokJn1x3EVGbp',
                                'https://youtu.be/ZJL4UGSbeFg?si=KbiBT-j4dTzYhJMZ',
                                'https://youtu.be/ur1cb_OztPQ?si=Njdwclw1OAkX0iK4', 
                                'https://youtu.be/9wQID0OG9yE?si=fS_femMdVikvlEAf',
                                'https://youtu.be/TTA2buWlNyM?si=z167DOwcsK6l6-s7',
                                'https://youtu.be/eM213aMKTHg?si=oxj5bnqOmwmDHQlw',
                                'https://youtu.be/g7f6HiQ2LuU?si=W67vvQfWEJvgwMQ6',
                                'https://youtu.be/PKpQRjj_WbU?si=klaDYws7qwdn65oe',
                                'https://youtu.be/CpkZfkxdj2c?si=l8xbrNFumdjBAcJp']
                  }

disco_dict = {'file_name': ['GloriaGaynor.Iwillsurvive',
                            'RickJames.SuperFreak',
                            'VillagePeople.YMCA',
                            'MichaelJackson.BillieJean',
                            'DonnaSummer.Lovetoloveyoubaby', 
                            'Sylvester.Doyouwantfunk', 
                            'BeeGees.NightFever',
                            'JackieMoore.Thistimebaby',
                            'VanMcCoy.Thehustle',
                            'FranceJoli.Cometome',
                            'KoolNTheGang.Celebration'],
                  'file_url' : ['https://youtu.be/Faf1ch7Q9XE?si=CPmwy-9G-kVrGdmC',
                                'https://youtu.be/QYHxGBH6o4M?si=I6t2mWiZu7DY4dpn',
                                'https://youtu.be/CS9OO0S5w2k?si=6ohTz3XKVSIw_cUv',
                                'https://youtu.be/Zi_XLOBDo_Y?si=Yk51e9zrmVppZadb',
                                'https://youtu.be/z5SPIjlfMXo?si=i5VxX6fFbzeaX4oG', 
                                'https://youtu.be/5H69xXQRSgM?si=sluI-3cvxcQTcoge', 
                                'https://youtu.be/-ihs-vT9T3Q?si=a9aXAmPJIGEPN6P7',
                                'https://youtu.be/koUAtkI2aB0?si=IXjWcsNqejYu8dgU',
                                'https://youtu.be/wj23_nDFSfE?si=NuZD72dqoUq1zsXK',
                                'https://youtu.be/vcIXBJJeozM?si=w1-UTcNCh31BBoWK',
                                'https://youtu.be/3GwjfUFyY6M?si=Oda8yBhKr7PC7Of7']
                  }

hiphop_dict = {'file_name': ['2Pac.HitEmUp',
                             'TheNotorious.Juicy',
                             'Nas.NYStateofMind',
                             'DrDre.NuthinbutaGthang',
                             'MobbDeep.ShookOnes', 
                             'EricBRakim.PaidinFull', 
                             'Wu-TangClan.ProtectYaNeck',
                             'Eminem.LoseYourself',
                             'SnoopDogg.GinandJuice',
                             'Coolio.GangstaParadise',
                             'Future.KeepItBurnin'],
                  'file_url' : ['https://youtu.be/41qC3w3UUkU?si=maxtpz1bM9SbjTe_',
                                'https://youtu.be/_JZom_gVfuw?si=mrxcMSdD-v4ItUHA',
                                'https://youtu.be/hI8A14Qcv68?si=WqCVNyzqMlh6yukJ',
                                'https://youtu.be/8GliyDgAGQI?si=QlDWYxmpvvIgazVR',
                                'https://youtu.be/yoYZf-lBF_U?si=NnBSRnx9YcfsfM5G', 
                                'https://youtu.be/E7t8eoA_1jQ?si=FA8WePNk2B0h9fQJ', 
                                'https://youtu.be/R0IUR4gkPIE?si=6C6fqrJ4z2V_lFcn',
                                'https://youtu.be/_Yhyp-_hX2s?si=CPAXspv3xPakBxmw',
                                'https://youtu.be/fWCZse1iwE0?si=xJNeklzuIjM3Fx91',
                                'https://youtu.be/fPO76Jlnz6c?si=XZJorwfbCxBWK7Kl',
                                'https://youtu.be/fV5NzrTmqZM?si=yWbNflhkZwGEwjD2']
                  }

jazz_dict = {'file_name': ['MilesDavis.FreddieFreeloader',
                           'DaveBrubeck.TakeFive',
                           'ArtPepper.Youbesoniceto',
                           'JohnColtrane.Naima',
                           'CharlieBirbParker.YardbirdSuite', 
                           'OliverNelson.StolenMoments', 
                           'JimmySmith.WhenJohnnycomesmarching',
                           'ArtFarmer.KillerJoe',
                           'FreddieHubbard.LittleSunflow',
                           'MilesDavis.IfIWereABell',
                           'GrooveHolmes.Misty'],
                  'file_url' : ['https://youtu.be/ZZcuSBouhVA?si=EW_2ZqJ08SCwjXF4',
                                'https://youtu.be/vmDDOFXSgAs?si=TEvXmpv-T57VlFOM',
                                'https://youtu.be/xISaCzXYYg8?si=q3jqAxxpAIpX8izx',
                                'https://youtu.be/Cx-TxiBi43c?si=FS_nfrNUZMylf7kD',
                                'https://youtu.be/HmroWIcCNUI?si=zPhhuvR8Arhhi_OM', 
                                'https://youtu.be/I777BcgQL9o?si=D4H_rWBK3PD2HzSk', 
                                'https://youtu.be/KsAf0ra6Vd4?si=_wO-6rYNOjmRhIgn',
                                'https://youtu.be/u23Etcb-L9M?si=gY0WoTg8euAiGZF3',
                                'https://youtu.be/OtB8dEuEmNM?si=nOWLVBGTOR0tRvhJ',
                                'https://youtu.be/36wafFjFdYs?si=_SCOFJ1M9_vMQ3m2',
                                'https://youtu.be/1OKduoWWWsE?si=fl52PuNEyH8z3GCE']
                  }

metal_dict = {'file_name': ['Pantera.Walk',
                            'Tool.Enema',
                            'Slipknot.WaitNBleed',
                            'BlackSabbath.Paranoid',
                            'Motorhead.AceofSpades', 
                            'KillswitchEngage.TheSignalFire', 
                            'IronMaiden.Runtothehills',
                            'AliceinChains.WeDieYoung',
                            'Korn.Freakonaleash',
                            'Venom.Welcometohell',
                            'Metallica.FadetoBlack'],
                  'file_url' : ['https://youtu.be/AkFqg5wAuFk?si=tZiP70HseFgZh76m',
                                'https://youtu.be/CehYA3omb5o?si=W47xgQ-5c3-WSBls',
                                'https://youtu.be/B1zCN0YhW1s?si=g9V-javjqgolkNC1',
                                'https://youtu.be/BOTIIw76qiE?si=9JtBqAgrNnWimLk-',
                                'https://youtu.be/3mbvWn1EY6g?si=xrkPdIZJ1hgF014J', 
                                'https://youtu.be/2N0ShfOOEq4?si=7QSMVRw-AdfCmNwI', 
                                'https://youtu.be/86URGgqONvA?si=kxBj8JHTfoUAA3Dt',
                                'https://youtu.be/6JA25BIxgtk?si=eeeDIdYw_rHixzkU',
                                'https://youtu.be/jRGrNDV2mKc?si=_ZoHngoH5VO3gWlG',
                                'https://youtu.be/y3Whz4iBYqg?si=8JuKAGpEIV1ncwfS',
                                'https://youtu.be/WEQnzs8wl6E?si=OHwI5aon2tI2y4Dc']
                  }

pop_dict = {'file_name': ['EdSheeran.ShapeofYou',
                          'Adele.EasyonMe',
                          'LewisCapaldi.Someoneyouloved',
                          'ShawnMendesCamilaCabello.Senorita',
                          'MileyCyrus.Flowers', 
                          'CharliePuth.WeDonttalkanymore', 
                          'DuaLipa.NewRules',
                          'BrunoMars.Thelazysong',
                          'Ladygaga.Pokerface',
                          'Passenger.Lethergo',
                          'JustinBieber.Ghost'],
                  'file_url' : ['https://youtu.be/JGwWNGJdvx8?si=VetofNFjfvs19YPC',
                                'https://youtu.be/BR89eJVJrWs?si=hR4p825hJMXZiHrU',
                                'https://youtu.be/zABLecsR5UE?si=y3puVu6o4ZldXm-s',
                                'https://youtu.be/Pkh8UtuejGw?si=wrj4jNPB7I2tY308',
                                'https://youtu.be/G7KNmW9a75Y?si=fhYrdVqux-tCEfXY', 
                                'https://youtu.be/3AtDnEC4zak?si=GyW-1OsqYMwZvcZt', 
                                'https://youtu.be/k2qgadSvNyU?si=zusIMko_qq0ouAgi',
                                'https://youtu.be/fLexgOxsZu0?si=XuhyuTnvxbNvFgDu',
                                'https://youtu.be/bESGLojNYSo?si=drU_ddUiiSACdbKG',
                                'https://youtu.be/RBumgq5yVrA?si=JUxfQEPQN_zrYxJF',
                                'https://youtu.be/Fp8msa5uYsc?si=RZ4W8qb9dlBWnEVE']
                  }

reggae_dict = {'file_name': ['BujuBanto.Coconutwata',
                             'Raggalax.Climbingtothetop',
                             'StickFigure.EdgeoftheOcean',
                             'Topgun.Somebodytolove',
                             'Sublime.PawnShop', 
                             'Culture.Whyamirastaman', 
                             'BobMarley.Zimbabwe',
                             'PeterTosh.SteppingRazor',
                             'Yellowman.Sensemilla',
                             'PeterTosh.BushDoctor',
                             'Gregory.Isaacs.NightNurse'],
                  'file_url' : ['https://youtu.be/GA2cjipnkQ0?si=p2DUUJ1hW_wqwWUj',
                                'https://youtu.be/1GZsRW5Bp5o?si=iYUBMN6KM8px5Ts2',
                                'https://youtu.be/KUsf6PgD1_4?si=RsD76QFmPPylL4Ie',
                                'https://youtu.be/2G4v6igHs38?si=YPxXi5fFxhJK72Vx',
                                'https://youtu.be/gAQ7fGXHFJ8?si=hnYwhQwYVg2DXVz_', 
                                'https://youtu.be/YSzXRgvqkps?si=MPtVKivDHc4T_DeP', 
                                'https://youtu.be/tm9KHjxSzxw?si=oHJDcs5oWpNVVa4w',
                                'https://youtu.be/aLJFRgE4Ywk?si=rvUcH4RWYun-0-5L',
                                'https://youtu.be/8PqA0ekm1R8?si=VS2Qtifnk_1lO8fv',
                                'https://youtu.be/XcBnaA3LpZ4?si=9sWOrbXR-zjCRQrb',
                                'https://youtu.be/6-3TrCC9v8w?si=bIEqNd1VUeP-f1w5']
                  }

rock_dict = {'file_name': ['LinkinPark.Numb',
                           'GunsNRoses.NovemberRain',
                           'GreenDay.BoulevardofBrokenDreams',
                           'Queen.BohemianRhapsody',
                           'ArticMonkeys.DoIwannaknow', 
                           'TheCranberries.Zombie', 
                           'ACDC.Thunderstruck',
                           'BonJovi.ItsmyLife',
                           'RedHotChillipeppers.Californication',
                           'Radiohead.Creep',
                           'BreakingBenjamin.Breath'],
                  'file_url' : ['https://youtu.be/kXYiU_JCYtU?si=nSyVjDTIaAymSeMm',
                                'https://youtu.be/8SbUC-UaAxE?si=90qQ_bR88elBf0aw',
                                'https://youtu.be/Soa3gO7tL-c?si=yKuRqKoeMWN8_NTn',
                                'https://youtu.be/fJ9rUzIMcZQ?si=SLF4eubc8gOCag5K',
                                'https://youtu.be/bpOSxM0rNPM?si=7XHc77eszM8zgjHg', 
                                'https://youtu.be/6Ejga4kJUts?si=2N96mx7yuGtvrqCC', 
                                'https://youtu.be/v2AC41dglnM?si=yIL_Yt1NmHDHF2m2',
                                'https://youtu.be/vx2u5uUu3DE?si=pwchbZOSpgBhiukN',
                                'https://youtu.be/YlUKcNNmywk?si=zEi_Ta1PMiOdnl9Y',
                                'https://youtu.be/XFkzRNyygfk?si=DtNu-0l5T91zQ-J9',
                                'https://youtu.be/XlqkG4XD2ok?si=R622n3ubi0UW62I6']
                  }



In [ ]:
## downloading files in dicts 
for i in range(0, len(rock_dict['file_name'])):
    # get_youtube_dl_audio_file('classical', classical_dict['file_url'][i], classical_dict['file_name'][i])
    
    # get_youtube_dl_audio_file('blues', blues_dict['file_url'][i], blues_dict['file_name'][i])
    
    # get_youtube_dl_audio_file('country', country_dict['file_url'][i], country_dict['file_name'][i])
    
    # get_youtube_dl_audio_file('disco', disco_dict['file_url'][i], disco_dict['file_name'][i])
    
    # get_youtube_dl_audio_file('hiphop', hiphop_dict['file_url'][i], hiphop_dict['file_name'][i])
    
    # get_youtube_dl_audio_file('jazz', jazz_dict['file_url'][i], jazz_dict['file_name'][i])
    
    # get_youtube_dl_audio_file('metal', metal_dict['file_url'][i], metal_dict['file_name'][i])
    
    # get_youtube_dl_audio_file('pop', pop_dict['file_url'][i], pop_dict['file_name'][i])
    
    # get_youtube_dl_audio_file('reggae', reggae_dict['file_url'][i], reggae_dict['file_name'][i])
    
    get_youtube_dl_audio_file('rock', rock_dict['file_url'][i], rock_dict['file_name'][i])
    

In [ ]:
## converting from mp4 to wav Classical
import os

command2mp3 = "ffmpeg -i classical.Bach.ToccataNFugue.mp4 classical.Bach.ToccataNFugue.mp3"
command2wav = "ffmpeg -i classical.Bach.ToccataNFugue.mp3 classical.Bach.ToccataNFugue.wav"
os.system(command2mp3)
os.system("rm classical.Bach.ToccataNFugue.mp4")
os.system(command2wav)
os.system("rm classical.Bach.ToccataNFugue.mp3")


command2mp3 = "ffmpeg -i classical.FrederocChopin.FuneralMarch.mp4 classical.FrederocChopin.FuneralMarch.mp3"
command2wav = "ffmpeg -i classical.FrederocChopin.FuneralMarch.mp3 classical.FrederocChopin.FuneralMarch.wav"
os.system(command2mp3)
os.system("rm classical.FrederocChopin.FuneralMarch.mp4")
os.system(command2wav)
os.system("rm classical.FrederocChopin.FuneralMarch.mp3")

command2mp3 = "ffmpeg -i classical.HandelMessiah.hallelujah.mp4 classical.HandelMessiah.hallelujah.mp3"
command2wav = "ffmpeg -i classical.HandelMessiah.hallelujah.mp3 classical.HandelMessiah.hallelujah.wav"
os.system(command2mp3)
os.system("rm classical.HandelMessiah.hallelujah.mp4")
os.system(command2wav)
os.system("rm classical.HandelMessiah.hallelujah.mp3")

command2mp3 = "ffmpeg -i classical.Tchaikovsky.PianoN1.mp4 classical.Tchaikovsky.PianoN1.mp3"
command2wav = "ffmpeg -i classical.Tchaikovsky.PianoN1.mp3 classical.Tchaikovsky.PianoN1.wav"
os.system(command2mp3)
os.system("rm classical.Tchaikovsky.PianoN1.mp4")
os.system(command2wav)
os.system("rm classical.Tchaikovsky.PianoN1.mp3")

command2mp3 = "ffmpeg -i classical.Mozart.LNDF.mp4 classical.Mozart.LNDF.mp3"
command2wav = "ffmpeg -i classical.Mozart.LNDF.mp3 classical.Mozart.LNDF.wav"
os.system(command2mp3)
os.system("rm classical.Mozart.LNDF.mp4")
os.system(command2wav)
os.system("rm classical.Mozart.LNDF.mp3")

command2mp3 = "ffmpeg -i classical.Greig.MountKing.mp4 classical.Greig.MountKing.mp3"
command2wav = "ffmpeg -i classical.Greig.MountKing.mp3 classical.Greig.MountKing.wav"
os.system(command2mp3)
os.system('rm classical.Greig.MountKing.mp4')
os.system(command2wav)
os.system('rm classical.Greig.MountKing.mp3')

command2mp3 = "ffmpeg -i classical.Elgar.MarchPomp.mp4 classical.Elgar.MarchPomp.mp3"
command2wav = "ffmpeg -i classical.Elgar.MarchPomp.mp3 classical.Elgar.MarchPomp.wav"
os.system(command2mp3)
os.system("rm classical.Elgar.MarchPomp.mp4")
os.system(command2wav)
os.system("rm classical.Elgar.MarchPomp.mp3")

command2mp3 = "ffmpeg -i classical.Schubert.MMN1.mp4 classical.Schubert.MMN1.mp3"
command2wav = "ffmpeg -i classical.Schubert.MMN1.mp3 classical.Schubert.MMN1.wav"
os.system(command2mp3)
os.system("rm classical.Schubert.MMN1.mp4")
os.system(command2wav)
os.system("rm classical.Schubert.MMN1.mp3")

command2mp3 = "ffmpeg -i classical.Rossini.BarberSevilla.mp4 classical.Rossini.BarberSevilla.mp3"
command2wav = "ffmpeg -i classical.Rossini.BarberSevilla.mp3 classical.Rossini.BarberSevilla.wav"
os.system(command2mp3)
os.system("rm classical.Rossini.BarberSevilla.mp4")
os.system(command2wav)
os.system("rm classical.Rossini.BarberSevilla.mp3")

command2mp3 = "ffmpeg -i classical.Puccini.NessumDorma.mp4 classical.Puccini.NessumDorma.mp3"
command2wav = "ffmpeg -i classical.Puccini.NessumDorma.mp3 classical.Puccini.NessumDorma.wav"
os.system(command2mp3)
os.system("rm classical.Puccini.NessumDorma.mp4")
os.system(command2wav)
os.system("rm classical.Puccini.NessumDorma.mp3")

command2mp3 = "ffmpeg -i classical.Beethoven.PianokonsertN5Bach.TOFStravinsky.Petrushka.mp4 classical.Beethoven.PianokonsertN5Bach.TOFStravinsky.Petrushka.mp3"
command2wav = "ffmpeg -i classical.Beethoven.PianokonsertN5Bach.TOFStravinsky.Petrushka.mp3 classical.Beethoven.PianokonsertN5Bach.TOFStravinsky.Petrushka.wav"
os.system(command2mp3)
os.system("rm classical.Beethoven.PianokonsertN5Bach.TOFStravinsky.Petrushka.mp4")
os.system(command2wav)
os.system("rm classical.Beethoven.PianokonsertN5Bach.TOFStravinsky.Petrushka.mp3")

In [ ]:
## converting from mp4 to wav blues
import os

command2mp3 = "ffmpeg -i blues.Andra.AtLast.mp4 blues.Andra.AtLast.mp3"
command2wav = "ffmpeg -i blues.Andra.AtLast.mp3 blues.Andra.AtLast.wav"
os.system(command2mp3)
os.system("rm blues.Andra.AtLast.mp4")
os.system(command2wav)
os.system("rm blues.Andra.AtLast.mp3")

command2mp3 = "ffmpeg -i blues.AnnieLennoxIPutASpellOnYou.mp4 blues.AnnieLennoxIPutASpellOnYou.mp3"
command2wav = "ffmpeg -i blues.AnnieLennoxIPutASpellOnYou.mp3 blues.AnnieLennoxIPutASpellOnYou.wav"
os.system(command2mp3)
os.system("rm blues.AnnieLennoxIPutASpellOnYou.mp4")
os.system(command2wav)
os.system("rm blues.AnnieLennoxIPutASpellOnYou.mp3")

command2mp3 = "ffmpeg -i blues.BuddyGuy.FleshNBone.mp4 blues.BuddyGuy.FleshNBone.mp3"
command2wav = "ffmpeg -i blues.BuddyGuy.FleshNBone.mp3 blues.BuddyGuy.FleshNBone.wav"
os.system(command2mp3)
os.system("rm blues.BuddyGuy.FleshNBone.mp4")
os.system(command2wav)
os.system("rm blues.BuddyGuy.FleshNBone.mp3")

command2mp3 = "ffmpeg -i blues.DanielCastro.IllPlayTheBluesForYou.mp4 blues.DanielCastro.IllPlayTheBluesForYou.mp3"
command2wav = "ffmpeg -i blues.DanielCastro.IllPlayTheBluesForYou.mp3 blues.DanielCastro.IllPlayTheBluesForYou.wav"
os.system(command2mp3)
os.system("rm blues.DanielCastro.IllPlayTheBluesForYou.mp4")
os.system(command2wav)
os.system("rm blues.DanielCastro.IllPlayTheBluesForYou.mp3")

command2mp3 = "ffmpeg -i blues.DavidDeacon.TheyAllSangAlong.mp4 blues.DavidDeacon.TheyAllSangAlong.mp3"
command2wav = "ffmpeg -i blues.DavidDeacon.TheyAllSangAlong.mp3 blues.DavidDeacon.TheyAllSangAlong.wav"
os.system(command2mp3)
os.system("rm blues.DavidDeacon.TheyAllSangAlong.mp4")
os.system(command2wav)
os.system("rm blues.DavidDeacon.TheyAllSangAlong.mp3")

command2mp3 = "ffmpeg -i blues.GaryColeman.TheSkyisCrying.mp4 blues.GaryColeman.TheSkyisCrying.mp3"
command2wav = "ffmpeg -i blues.GaryColeman.TheSkyisCrying.mp3 blues.GaryColeman.TheSkyisCrying.wav"
os.system(command2mp3)
os.system("rm blues.GaryColeman.TheSkyisCrying.mp4")
os.system(command2wav)
os.system("rm blues.GaryColeman.TheSkyisCrying.mp3")

command2mp3 = "ffmpeg -i blues.RichardThompson.IfLoveWhispersYour.mp4 blues.RichardThompson.IfLoveWhispersYour.mp3"
command2wav = "ffmpeg -i blues.RichardThompson.IfLoveWhispersYour.mp3 blues.RichardThompson.IfLoveWhispersYour.wav"
os.system(command2mp3)
os.system("rm blues.RichardThompson.IfLoveWhispersYour.mp4")
os.system(command2wav)
os.system("rm blues.RichardThompson.IfLoveWhispersYour.mp3")

command2mp3 = "ffmpeg -i blues.StevieRayVaughan.LittleWing.mp4 blues.StevieRayVaughan.LittleWing.mp3"
command2wav = "ffmpeg -i blues.StevieRayVaughan.LittleWing.mp3 blues.StevieRayVaughan.LittleWing.wav"
os.system(command2mp3)
os.system("rm blues.StevieRayVaughan.LittleWing.mp4")
os.system(command2wav)
os.system("rm blues.StevieRayVaughan.LittleWing.mp3")

command2mp3 = "ffmpeg -i blues.TheBarrBrothers.HalfCrazy.mp4 blues.TheBarrBrothers.HalfCrazy.mp3"
command2wav = "ffmpeg -i blues.TheBarrBrothers.HalfCrazy.mp3 blues.TheBarrBrothers.HalfCrazy.wav"
os.system(command2mp3)
os.system("rm blues.TheBarrBrothers.HalfCrazy.mp4")
os.system(command2wav)
os.system("rm blues.TheBarrBrothers.HalfCrazy.mp3")

command2mp3 = "ffmpeg -i blues.TheBarrBrothers.LordIJustCant.mp4 blues.TheBarrBrothers.LordIJustCant.mp3"
command2wav = "ffmpeg -i blues.TheBarrBrothers.LordIJustCant.mp3 blues.TheBarrBrothers.LordIJustCant.wav"
os.system(command2mp3)
os.system("rm blues.TheBarrBrothers.LordIJustCant.mp4")
os.system(command2wav)
os.system("rm blues.TheBarrBrothers.LordIJustCant.mp3")

command2mp3 = "ffmpeg -i blues.WrecklessStrangers.FastGirls.mp4 blues.WrecklessStrangers.FastGirls.mp3"
command2wav = "ffmpeg -i blues.WrecklessStrangers.FastGirls.mp3 blues.WrecklessStrangers.FastGirls.wav"
os.system(command2mp3)
os.system("rm blues.WrecklessStrangers.FastGirls.mp4")
os.system(command2wav)
os.system("rm blues.WrecklessStrangers.FastGirls.mp3")



In [ ]:
## converting from mp4 to wav country
import os

command2mp3 = "ffmpeg -i country.AlanJackson.RememberWhen.mp4 country.AlanJackson.RememberWhen.mp3"
command2wav = "ffmpeg -i country.AlanJackson.RememberWhen.mp3 country.AlanJackson.RememberWhen.wav"
os.system(command2mp3)
os.system("rm country.AlanJackson.RememberWhen.mp4")
os.system(command2wav)
os.system("rm country.AlanJackson.RememberWhen.mp3")

command2mp3 = "ffmpeg -i country.BillliCurrington.PeopleareCrazy.mp4 country.BillliCurrington.PeopleareCrazy.mp3"
command2wav = "ffmpeg -i country.BillliCurrington.PeopleareCrazy.mp3 country.BillliCurrington.PeopleareCrazy.wav"
os.system(command2mp3)
os.system("rm country.BillliCurrington.PeopleareCrazy.mp4")
os.system(command2wav)
os.system("rm country.BillliCurrington.PeopleareCrazy.mp3")

command2mp3 = "ffmpeg -i country.ChaseMatthew.WeAllGrowUp.mp4 country.ChaseMatthew.WeAllGrowUp.mp3"
command2wav = "ffmpeg -i country.ChaseMatthew.WeAllGrowUp.mp3 country.ChaseMatthew.WeAllGrowUp.wav"
os.system(command2mp3)
os.system("rm country.ChaseMatthew.WeAllGrowUp.mp4")
os.system(command2wav)
os.system("rm country.ChaseMatthew.WeAllGrowUp.mp3")

command2mp3 = "ffmpeg -i country.CodyJohnson.Thepainter.mp4 country.CodyJohnson.Thepainter.mp3"
command2wav = "ffmpeg -i country.CodyJohnson.Thepainter.mp3 country.CodyJohnson.Thepainter.wav"
os.system(command2mp3)
os.system("rm country.CodyJohnson.Thepainter.mp4")
os.system(command2wav)
os.system("rm country.CodyJohnson.Thepainter.mp3")

command2mp3 = "ffmpeg -i country.DariusRucker.WagonWheel.mp4 country.DariusRucker.WagonWheel.mp3"
command2wav = "ffmpeg -i country.DariusRucker.WagonWheel.mp3 country.DariusRucker.WagonWheel.wav"
os.system(command2mp3)
os.system("rm country.DariusRucker.WagonWheel.mp4")
os.system(command2wav)
os.system("rm country.DariusRucker.WagonWheel.mp3")

command2mp3 = "ffmpeg -i country.KennyChesney.TakeHerHome.mp4 country.KennyChesney.TakeHerHome.mp3"
command2wav = "ffmpeg -i country.KennyChesney.TakeHerHome.mp3 country.KennyChesney.TakeHerHome.wav"
os.system(command2mp3)
os.system("rm country.KennyChesney.TakeHerHome.mp4")
os.system(command2wav)
os.system("rm country.KennyChesney.TakeHerHome.mp3")

command2mp3 = "ffmpeg -i country.LadyAntebellum.NeedYouNow.mp4 country.LadyAntebellum.NeedYouNow.mp3"
command2wav = "ffmpeg -i country.LadyAntebellum.NeedYouNow.mp3 country.LadyAntebellum.NeedYouNow.wav"
os.system(command2mp3)
os.system("rm country.LadyAntebellum.NeedYouNow.mp4")
os.system(command2wav)
os.system("rm country.LadyAntebellum.NeedYouNow.mp3")

command2mp3 = "ffmpeg -i country.LeeBrice.OneofThemGirls.mp4 country.LeeBrice.OneofThemGirls.mp3"
command2wav = "ffmpeg -i country.LeeBrice.OneofThemGirls.mp3 country.LeeBrice.OneofThemGirls.wav"
os.system(command2mp3)
os.system("rm country.LeeBrice.OneofThemGirls.mp4")
os.system(command2wav)
os.system("rm country.LeeBrice.OneofThemGirls.mp3")

command2mp3 = "ffmpeg -i country.LukeCombs.BeautifulCrazy.mp4 country.LukeCombs.BeautifulCrazy.mp3"
command2wav = "ffmpeg -i country.LukeCombs.BeautifulCrazy.mp3 country.LukeCombs.BeautifulCrazy.wav"
os.system(command2mp3)
os.system("rm country.LukeCombs.BeautifulCrazy.mp4")
os.system(command2wav)
os.system("rm country.LukeCombs.BeautifulCrazy.mp3")

command2mp3 = "ffmpeg -i country.Midlan.DrinkinProblem.mp4 country.Midlan.DrinkinProblem.mp3"
command2wav = "ffmpeg -i country.Midlan.DrinkinProblem.mp3 country.Midlan.DrinkinProblem.wav"
os.system(command2mp3)
os.system("rm country.Midlan.DrinkinProblem.mp4")
os.system(command2wav)
os.system("rm country.Midlan.DrinkinProblem.mp3")

command2mp3 = "ffmpeg -i country.ShaniaTwain.Ifeellikeawoman.mp4 country.ShaniaTwain.Ifeellikeawoman.mp3"
command2wav = "ffmpeg -i country.ShaniaTwain.Ifeellikeawoman.mp3 country.ShaniaTwain.Ifeellikeawoman.wav"
os.system(command2mp3)
os.system("rm country.ShaniaTwain.Ifeellikeawoman.mp4")
os.system(command2wav)
os.system("rm country.ShaniaTwain.Ifeellikeawoman.mp3")

In [ ]:
## converting from mp4 to wav disco
import os

command2mp3 = "ffmpeg -i disco.BeeGees.NightFever.mp4 disco.BeeGees.NightFever.mp3"
command2wav = "ffmpeg -i disco.BeeGees.NightFever.mp3 disco.BeeGees.NightFever.wav"
os.system(command2mp3)
os.system("rm disco.BeeGees.NightFever.mp4")
os.system(command2wav)
os.system("rm disco.BeeGees.NightFever.mp3")

command2mp3 = "ffmpeg -i disco.DonnaSummer.Lovetoloveyoubaby.mp4 disco.DonnaSummer.Lovetoloveyoubaby.mp3"
command2wav = "ffmpeg -i disco.DonnaSummer.Lovetoloveyoubaby.mp3 disco.DonnaSummer.Lovetoloveyoubaby.wav"
os.system(command2mp3)
os.system("rm disco.DonnaSummer.Lovetoloveyoubaby.mp4")
os.system(command2wav)
os.system("rm disco.DonnaSummer.Lovetoloveyoubaby.mp3")

command2mp3 = "ffmpeg -i disco.FranceJoli.Cometome.mp4 disco.FranceJoli.Cometome.mp3"
command2wav = "ffmpeg -i disco.FranceJoli.Cometome.mp3 disco.FranceJoli.Cometome.wav"
os.system(command2mp3)
os.system("rm disco.FranceJoli.Cometome.mp4")
os.system(command2wav)
os.system("rm disco.FranceJoli.Cometome.mp3")

command2mp3 = "ffmpeg -i disco.GloriaGaynor.Iwillsurvive.mp4 disco.GloriaGaynor.Iwillsurvive.mp3"
command2wav = "ffmpeg -i disco.GloriaGaynor.Iwillsurvive.mp3 disco.GloriaGaynor.Iwillsurvive.wav"
os.system(command2mp3)
os.system("rm disco.GloriaGaynor.Iwillsurvive.mp4")
os.system(command2wav)
os.system("rm disco.GloriaGaynor.Iwillsurvive.mp3")

command2mp3 = "ffmpeg -i disco.JackieMoore.Thistimebaby.mp4 disco.JackieMoore.Thistimebaby.mp3"
command2wav = "ffmpeg -i disco.JackieMoore.Thistimebaby.mp3 disco.JackieMoore.Thistimebaby.wav"
os.system(command2mp3)
os.system("rm disco.JackieMoore.Thistimebaby.mp4")
os.system(command2wav)
os.system("rm disco.JackieMoore.Thistimebaby.mp3")

command2mp3 = "ffmpeg -i disco.KoolNTheGang.Celebration.mp4 disco.KoolNTheGang.Celebration.mp3"
command2wav = "ffmpeg -i disco.KoolNTheGang.Celebration.mp3 disco.KoolNTheGang.Celebration.wav"
os.system(command2mp3)
os.system("rm disco.KoolNTheGang.Celebration.mp4")
os.system(command2wav)
os.system("rm disco.KoolNTheGang.Celebration.mp3")

command2mp3 = "ffmpeg -i disco.MichaelJackson.BillieJean.mp4 disco.MichaelJackson.BillieJean.mp3"
command2wav = "ffmpeg -i disco.MichaelJackson.BillieJean.mp3 disco.MichaelJackson.BillieJean.wav"
os.system(command2mp3)
os.system("rm disco.MichaelJackson.BillieJean.mp4")
os.system(command2wav)
os.system("rm disco.MichaelJackson.BillieJean.mp3")

command2mp3 = "ffmpeg -i disco.RickJames.SuperFreak.mp4 disco.RickJames.SuperFreak.mp3"
command2wav = "ffmpeg -i disco.RickJames.SuperFreak.mp3 disco.RickJames.SuperFreak.wav"
os.system(command2mp3)
os.system("rm disco.RickJames.SuperFreak.mp4")
os.system(command2wav)
os.system("rm disco.RickJames.SuperFreak.mp3")

command2mp3 = "ffmpeg -i disco.Sylvester.Doyouwantfunk.mp4 disco.Sylvester.Doyouwantfunk.mp3"
command2wav = "ffmpeg -i disco.Sylvester.Doyouwantfunk.mp3 disco.Sylvester.Doyouwantfunk.wav"
os.system(command2mp3)
os.system("rm disco.Sylvester.Doyouwantfunk.mp4")
os.system(command2wav)
os.system("rm disco.Sylvester.Doyouwantfunk.mp3")

command2mp3 = "ffmpeg -i disco.VanMcCoy.Thehustle.mp4 disco.VanMcCoy.Thehustle.mp3"
command2wav = "ffmpeg -i disco.VanMcCoy.Thehustle.mp3 disco.VanMcCoy.Thehustle.wav"
os.system(command2mp3)
os.system("rm disco.VanMcCoy.Thehustle.mp4")
os.system(command2wav)
os.system("rm disco.VanMcCoy.Thehustle.mp3")

command2mp3 = "ffmpeg -i disco.VillagePeople.YMCA.mp4 disco.VillagePeople.YMCA.mp3"
command2wav = "ffmpeg -i disco.VillagePeople.YMCA.mp3 disco.VillagePeople.YMCA.wav"
os.system(command2mp3)
os.system("rm disco.VillagePeople.YMCA.mp4")
os.system(command2wav)
os.system("rm disco.VillagePeople.YMCA.mp3")

In [ ]:
## converting from mp4 to wav hiphop
import os

command2mp3 = "ffmpeg -i hiphop.2Pac.HitEmUp.mp4 hiphop.2Pac.HitEmUp.mp3"
command2wav = "ffmpeg -i hiphop.2Pac.HitEmUp.mp3 hiphop.2Pac.HitEmUp.wav"
os.system(command2mp3)
os.system("rm hiphop.2Pac.HitEmUp.mp4")
os.system(command2wav)
os.system("rm hiphop.2Pac.HitEmUp.mp3")

command2mp3 = "ffmpeg -i hiphop.Coolio.GangstaParadise.mp4 hiphop.Coolio.GangstaParadise.mp3"
command2wav = "ffmpeg -i hiphop.Coolio.GangstaParadise.mp3 hiphop.Coolio.GangstaParadise.wav"
os.system(command2mp3)
os.system("rm hiphop.Coolio.GangstaParadise.mp4")
os.system(command2wav)
os.system("rm hiphop.Coolio.GangstaParadise.mp3")

command2mp3 = "ffmpeg -i hiphop.DrDre.NuthinbutaGthang.mp4 hiphop.DrDre.NuthinbutaGthang.mp3"
command2wav = "ffmpeg -i hiphop.DrDre.NuthinbutaGthang.mp3 hiphop.DrDre.NuthinbutaGthang.wav"
os.system(command2mp3)
os.system("rm hiphop.DrDre.NuthinbutaGthang.mp4")
os.system(command2wav)
os.system("rm hiphop.DrDre.NuthinbutaGthang.mp3")

command2mp3 = "ffmpeg -i hiphop.Eminem.LoseYourself.mp4 hiphop.Eminem.LoseYourself.mp3"
command2wav = "ffmpeg -i hiphop.Eminem.LoseYourself.mp3 hiphop.Eminem.LoseYourself.wav"
os.system(command2mp3)
os.system("rm hiphop.Eminem.LoseYourself.mp4")
os.system(command2wav)
os.system("rm hiphop.Eminem.LoseYourself.mp3")

command2mp3 = "ffmpeg -i hiphop.EricBRakim.PaidinFull.mp4 hiphop.EricBRakim.PaidinFull.mp3"
command2wav = "ffmpeg -i hiphop.EricBRakim.PaidinFull.mp3 hiphop.EricBRakim.PaidinFull.wav"
os.system(command2mp3)
os.system("rm hiphop.EricBRakim.PaidinFull.mp4")
os.system(command2wav)
os.system("rm hiphop.EricBRakim.PaidinFull.mp3")

command2mp3 = "ffmpeg -i hiphop.Future.KeepItBurnin.mp4 hiphop.Future.KeepItBurnin.mp3"
command2wav = "ffmpeg -i hiphop.Future.KeepItBurnin.mp3 hiphop.Future.KeepItBurnin.wav"
os.system(command2mp3)
os.system("rm hiphop.Future.KeepItBurnin.mp4")
os.system(command2wav)
os.system("rm hiphop.Future.KeepItBurnin.mp3")

command2mp3 = "ffmpeg -i hiphop.MobbDeep.ShookOnes.mp4 hiphop.MobbDeep.ShookOnes.mp3"
command2wav = "ffmpeg -i hiphop.MobbDeep.ShookOnes.mp3 hiphop.MobbDeep.ShookOnes.wav"
os.system(command2mp3)
os.system("rm hiphop.MobbDeep.ShookOnes.mp4")
os.system(command2wav)
os.system("rm hiphop.MobbDeep.ShookOnes.mp3")

command2mp3 = "ffmpeg -i hiphop.Nas.NYStateofMind.mp4 hiphop.Nas.NYStateofMind.mp3"
command2wav = "ffmpeg -i hiphop.Nas.NYStateofMind.mp3 hiphop.Nas.NYStateofMind.wav"
os.system(command2mp3)
os.system("rm hiphop.Nas.NYStateofMind.mp4")
os.system(command2wav)
os.system("rm hiphop.Nas.NYStateofMind.mp3")

command2mp3 = "ffmpeg -i hiphop.SnoopDogg.GinandJuice.mp4 hiphop.SnoopDogg.GinandJuice.mp3"
command2wav = "ffmpeg -i hiphop.SnoopDogg.GinandJuice.mp3 hiphop.SnoopDogg.GinandJuice.wav"
os.system(command2mp3)
os.system("rm hiphop.SnoopDogg.GinandJuice.mp4")
os.system(command2wav)
os.system("rm hiphop.SnoopDogg.GinandJuice.mp3")

command2mp3 = "ffmpeg -i hiphop.TheNotorious.Juicy.mp4 hiphop.TheNotorious.Juicy.mp3"
command2wav = "ffmpeg -i hiphop.TheNotorious.Juicy.mp3 hiphop.TheNotorious.Juicy.wav"
os.system(command2mp3)
os.system("rm hiphop.TheNotorious.Juicy.mp4")
os.system(command2wav)
os.system("rm hiphop.TheNotorious.Juicy.mp3")

command2mp3 = "ffmpeg -i hiphop.Wu-TangClan.ProtectYaNeck.mp4 hiphop.Wu-TangClan.ProtectYaNeck.mp3"
command2wav = "ffmpeg -i hiphop.Wu-TangClan.ProtectYaNeck.mp3 hiphop.Wu-TangClan.ProtectYaNeck.wav"
os.system(command2mp3)
os.system("rm hiphop.Wu-TangClan.ProtectYaNeck.mp4")
os.system(command2wav)
os.system("rm hiphop.Wu-TangClan.ProtectYaNeck.mp3")


In [ ]:
## converting from mp4 to wav jazz
import os

command2mp3 = "ffmpeg -i jazz.ArtFarmer.KillerJoe.mp4 jazz.ArtFarmer.KillerJoe.mp3"
command2wav = "ffmpeg -i jazz.ArtFarmer.KillerJoe.mp3 jazz.ArtFarmer.KillerJoe.wav"
os.system(command2mp3)
os.system("rm jazz.ArtFarmer.KillerJoe.mp4")
os.system(command2wav)
os.system("rm jazz.ArtFarmer.KillerJoe.mp3")

command2mp3 = "ffmpeg -i jazz.ArtPepper.Youbesoniceto.mp4 jazz.ArtPepper.Youbesoniceto.mp3"
command2wav = "ffmpeg -i jazz.ArtPepper.Youbesoniceto.mp3 jazz.ArtPepper.Youbesoniceto.wav"
os.system(command2mp3)
os.system("rm jazz.ArtPepper.Youbesoniceto.mp4")
os.system(command2wav)
os.system("rm jazz.ArtPepper.Youbesoniceto.mp3")

command2mp3 = "ffmpeg -i jazz.CharlieBirbParker.YardbirdSuite.mp4 jazz.CharlieBirbParker.YardbirdSuite.mp3"
command2wav = "ffmpeg -i jazz.CharlieBirbParker.YardbirdSuite.mp3 jazz.CharlieBirbParker.YardbirdSuite.wav"
os.system(command2mp3)
os.system("rm jazz.CharlieBirbParker.YardbirdSuite.mp4")
os.system(command2wav)
os.system("rm jazz.CharlieBirbParker.YardbirdSuite.mp3")

command2mp3 = "ffmpeg -i jazz.DaveBrubeck.TakeFive.mp4 jazz.DaveBrubeck.TakeFive.mp3"
command2wav = "ffmpeg -i jazz.DaveBrubeck.TakeFive.mp3 jazz.DaveBrubeck.TakeFive.wav"
os.system(command2mp3)
os.system("rm jazz.DaveBrubeck.TakeFive.mp4")
os.system(command2wav)
os.system("rm jazz.DaveBrubeck.TakeFive.mp3")

command2mp3 = "ffmpeg -i jazz.FreddieHubbard.LittleSunflow.mp4 jazz.FreddieHubbard.LittleSunflow.mp3"
command2wav = "ffmpeg -i jazz.FreddieHubbard.LittleSunflow.mp3 jazz.FreddieHubbard.LittleSunflow.wav"
os.system(command2mp3)
os.system("rm jazz.FreddieHubbard.LittleSunflow.mp4")
os.system(command2wav)
os.system("rm jazz.FreddieHubbard.LittleSunflow.mp3")

command2mp3 = "ffmpeg -i jazz.GrooveHolmes.Misty.mp4 jazz.GrooveHolmes.Misty.mp3"
command2wav = "ffmpeg -i jazz.GrooveHolmes.Misty.mp3 jazz.GrooveHolmes.Misty.wav"
os.system(command2mp3)
os.system("rm jazz.GrooveHolmes.Misty.mp4")
os.system(command2wav)
os.system("rm jazz.GrooveHolmes.Misty.mp3")

command2mp3 = "ffmpeg -i jazz.JimmySmith.WhenJohnnycomesmarching.mp4 jazz.JimmySmith.WhenJohnnycomesmarching.mp3"
command2wav = "ffmpeg -i jazz.JimmySmith.WhenJohnnycomesmarching.mp3 jazz.JimmySmith.WhenJohnnycomesmarching.wav"
os.system(command2mp3)
os.system("rm jazz.JimmySmith.WhenJohnnycomesmarching.mp4")
os.system(command2wav)
os.system("rm jazz.JimmySmith.WhenJohnnycomesmarching.mp3")

command2mp3 = "ffmpeg -i jazz.JohnColtrane.Naima.mp4 jazz.JohnColtrane.Naima.mp3"
command2wav = "ffmpeg -i jazz.JohnColtrane.Naima.mp3 jazz.JohnColtrane.Naima.wav"
os.system(command2mp3)
os.system("rm jazz.JohnColtrane.Naima.mp4")
os.system(command2wav)
os.system("rm jazz.JohnColtrane.Naima.mp3")

command2mp3 = "ffmpeg -i jazz.MilesDavis.FreddieFreeloader.mp4 jazz.MilesDavis.FreddieFreeloader.mp3"
command2wav = "ffmpeg -i jazz.MilesDavis.FreddieFreeloader.mp3 jazz.MilesDavis.FreddieFreeloader.wav"
os.system(command2mp3)
os.system("rm jazz.MilesDavis.FreddieFreeloader.mp4")
os.system(command2wav)
os.system("rm jazz.MilesDavis.FreddieFreeloader.mp3")

command2mp3 = "ffmpeg -i jazz.MilesDavis.IfIWereABell.mp4 jazz.MilesDavis.IfIWereABell.mp3"
command2wav = "ffmpeg -i jazz.MilesDavis.IfIWereABell.mp3 jazz.MilesDavis.IfIWereABell.wav"
os.system(command2mp3)
os.system("rm jazz.MilesDavis.IfIWereABell.mp4")
os.system(command2wav)
os.system("rm jazz.MilesDavis.IfIWereABell.mp3")

command2mp3 = "ffmpeg -i jazz.OliverNelson.StolenMoments.mp4 jazz.OliverNelson.StolenMoments.mp3"
command2wav = "ffmpeg -i jazz.OliverNelson.StolenMoments.mp3 jazz.OliverNelson.StolenMoments.wav"
os.system(command2mp3)
os.system("rm jazz.OliverNelson.StolenMoments.mp4")
os.system(command2wav)
os.system("rm jazz.OliverNelson.StolenMoments.mp3")

In [ ]:
## converting from mp4 to wav metal
import os

command2mp3 = "ffmpeg -i metal.AliceinChains.WeDieYoung.mp4 metal.AliceinChains.WeDieYoung.mp3"
command2wav = "ffmpeg -i metal.AliceinChains.WeDieYoung.mp3 metal.AliceinChains.WeDieYoung.wav"
os.system(command2mp3)
os.system("rm metal.AliceinChains.WeDieYoung.mp4")
os.system(command2wav)
os.system("rm metal.AliceinChains.WeDieYoung.mp3")

command2mp3 = "ffmpeg -i metal.BlackSabbath.Paranoid.mp4 metal.BlackSabbath.Paranoid.mp3"
command2wav = "ffmpeg -i metal.BlackSabbath.Paranoid.mp3 metal.BlackSabbath.Paranoid.wav"
os.system(command2mp3)
os.system("rm metal.BlackSabbath.Paranoid.mp4")
os.system(command2wav)
os.system("rm metal.BlackSabbath.Paranoid.mp3")

command2mp3 = "ffmpeg -i metal.IronMaiden.Runtothehills.mp4 metal.IronMaiden.Runtothehills.mp3"
command2wav = "ffmpeg -i metal.IronMaiden.Runtothehills.mp3 metal.IronMaiden.Runtothehills.wav"
os.system(command2mp3)
os.system("rm metal.IronMaiden.Runtothehills.mp4")
os.system(command2wav)
os.system("rm metal.IronMaiden.Runtothehills.mp3")

command2mp3 = "ffmpeg -i metal.KillswitchEngage.TheSignalFire.mp4 metal.KillswitchEngage.TheSignalFire.mp3"
command2wav = "ffmpeg -i metal.KillswitchEngage.TheSignalFire.mp3 metal.KillswitchEngage.TheSignalFire.wav"
os.system(command2mp3)
os.system("rm metal.KillswitchEngage.TheSignalFire.mp4")
os.system(command2wav)
os.system("rm metal.KillswitchEngage.TheSignalFire.mp3")

command2mp3 = "ffmpeg -i metal.Korn.Freakonaleash.mp4 metal.Korn.Freakonaleash.mp3"
command2wav = "ffmpeg -i metal.Korn.Freakonaleash.mp3 metal.Korn.Freakonaleash.wav"
os.system(command2mp3)
os.system("rm metal.Korn.Freakonaleash.mp4")
os.system(command2wav)
os.system("rm metal.Korn.Freakonaleash.mp3")

command2mp3 = "ffmpeg -i metal.Metallica.FadetoBlack.mp4 metal.Metallica.FadetoBlack.mp3"
command2wav = "ffmpeg -i metal.Metallica.FadetoBlack.mp3 metal.Metallica.FadetoBlack.wav"
os.system(command2mp3)
os.system("rm metal.Metallica.FadetoBlack.mp4")
os.system(command2wav)
os.system("rm metal.Metallica.FadetoBlack.mp3")

command2mp3 = "ffmpeg -i metal.Motorhead.AceofSpades.mp4 metal.Motorhead.AceofSpades.mp3"
command2wav = "ffmpeg -i metal.Motorhead.AceofSpades.mp3 metal.Motorhead.AceofSpades.wav"
os.system(command2mp3)
os.system("rm metal.Motorhead.AceofSpades.mp4")
os.system(command2wav)
os.system("rm metal.Motorhead.AceofSpades.mp3")

command2mp3 = "ffmpeg -i metal.Pantera.Walk.mp4 metal.Pantera.Walk.mp3"
command2wav = "ffmpeg -i metal.Pantera.Walk.mp3 metal.Pantera.Walk.wav"
os.system(command2mp3)
os.system("rm metal.Pantera.Walk.mp4")
os.system(command2wav)
os.system("rm metal.Pantera.Walk.mp3")

command2mp3 = "ffmpeg -i metal.Slipknot.WaitNBleed.mp4 metal.Slipknot.WaitNBleed.mp3"
command2wav = "ffmpeg -i metal.Slipknot.WaitNBleed.mp3 metal.Slipknot.WaitNBleed.wav"
os.system(command2mp3)
os.system("rm metal.Slipknot.WaitNBleed.mp4")
os.system(command2wav)
os.system("rm metal.Slipknot.WaitNBleed.mp3")

command2mp3 = "ffmpeg -i metal.Tool.Enema.mp4 metal.Tool.Enema.mp3"
command2wav = "ffmpeg -i metal.Tool.Enema.mp3 metal.Tool.Enema.wav"
os.system(command2mp3)
os.system("rm metal.Tool.Enema.mp4")
os.system(command2wav)
os.system("rm metal.Tool.Enema.mp3")

command2mp3 = "ffmpeg -i metal.Venom.Welcometohell.mp4 metal.Venom.Welcometohell.mp3"
command2wav = "ffmpeg -i metal.Venom.Welcometohell.mp3 metal.Venom.Welcometohell.wav"
os.system(command2mp3)
os.system("rm metal.Venom.Welcometohell.mp4")
os.system(command2wav)
os.system("rm metal.Venom.Welcometohell.mp3")

In [ ]:
## converting from mp4 to wav pop
import os

command2mp3 = "ffmpeg -i pop.Adele.EasyonMe.mp4 pop.Adele.EasyonMe.mp3"
command2wav = "ffmpeg -i pop.Adele.EasyonMe.mp3 pop.Adele.EasyonMe.wav"
os.system(command2mp3)
os.system("rm pop.Adele.EasyonMe.mp4")
os.system(command2wav)
os.system("rm pop.Adele.EasyonMe.mp3")

command2mp3 = "ffmpeg -i pop.BrunoMars.Thelazysong.mp4 pop.BrunoMars.Thelazysong.mp3"
command2wav = "ffmpeg -i pop.BrunoMars.Thelazysong.mp3 pop.BrunoMars.Thelazysong.wav"
os.system(command2mp3)
os.system("rm pop.BrunoMars.Thelazysong.mp4")
os.system(command2wav)
os.system("rm pop.BrunoMars.Thelazysong.mp3")

command2mp3 = "ffmpeg -i pop.CharliePuth.WeDonttalkanymore.mp4 pop.CharliePuth.WeDonttalkanymore.mp3"
command2wav = "ffmpeg -i pop.CharliePuth.WeDonttalkanymore.mp3 pop.CharliePuth.WeDonttalkanymore.wav"
os.system(command2mp3)
os.system("rm pop.CharliePuth.WeDonttalkanymore.mp4")
os.system(command2wav)
os.system("rm pop.CharliePuth.WeDonttalkanymore.mp3")

command2mp3 = "ffmpeg -i pop.DuaLipa.NewRules.mp4 pop.DuaLipa.NewRules.mp3"
command2wav = "ffmpeg -i pop.DuaLipa.NewRules.mp3 pop.DuaLipa.NewRules.wav"
os.system(command2mp3)
os.system("rm pop.DuaLipa.NewRules.mp4")
os.system(command2wav)
os.system("rm pop.DuaLipa.NewRules.mp3")

command2mp3 = "ffmpeg -i pop.EdSheeran.ShapeofYou.mp4 pop.EdSheeran.ShapeofYou.mp3"
command2wav = "ffmpeg -i pop.EdSheeran.ShapeofYou.mp3 pop.EdSheeran.ShapeofYou.wav"
os.system(command2mp3)
os.system("rm pop.EdSheeran.ShapeofYou.mp4")
os.system(command2wav)
os.system("rm pop.EdSheeran.ShapeofYou.mp3")

command2mp3 = "ffmpeg -i pop.JustinBieber.Ghost.mp4 pop.JustinBieber.Ghost.mp3"
command2wav = "ffmpeg -i pop.JustinBieber.Ghost.mp3 pop.JustinBieber.Ghost.wav"
os.system(command2mp3)
os.system("rm pop.JustinBieber.Ghost.mp4")
os.system(command2wav)
os.system("rm pop.JustinBieber.Ghost.mp3")

command2mp3 = "ffmpeg -i pop.Ladygaga.Pokerface.mp4 pop.Ladygaga.Pokerface.mp3"
command2wav = "ffmpeg -i pop.Ladygaga.Pokerface.mp3 pop.Ladygaga.Pokerface.wav"
os.system(command2mp3)
os.system("rm pop.Ladygaga.Pokerface.mp4")
os.system(command2wav)
os.system("rm pop.Ladygaga.Pokerface.mp3")

command2mp3 = "ffmpeg -i pop.LewisCapaldi.Someoneyouloved.mp4 pop.LewisCapaldi.Someoneyouloved.mp3"
command2wav = "ffmpeg -i pop.LewisCapaldi.Someoneyouloved.mp3 pop.LewisCapaldi.Someoneyouloved.wav"
os.system(command2mp3)
os.system("rm pop.LewisCapaldi.Someoneyouloved.mp4")
os.system(command2wav)
os.system("rm pop.LewisCapaldi.Someoneyouloved.mp3")

command2mp3 = "ffmpeg -i pop.MileyCyrus.Flowers.mp4 pop.MileyCyrus.Flowers.mp3"
command2wav = "ffmpeg -i pop.MileyCyrus.Flowers.mp3 pop.MileyCyrus.Flowers.wav"
os.system(command2mp3)
os.system("rm pop.MileyCyrus.Flowers.mp4")
os.system(command2wav)
os.system("rm pop.MileyCyrus.Flowers.mp3")

command2mp3 = "ffmpeg -i pop.Passenger.Lethergo.mp4 pop.Passenger.Lethergo.mp3"
command2wav = "ffmpeg -i pop.Passenger.Lethergo.mp3 pop.Passenger.Lethergo.wav"
os.system(command2mp3)
os.system("rm pop.Passenger.Lethergo.mp4")
os.system(command2wav)
os.system("rm pop.Passenger.Lethergo.mp3")

command2mp3 = "ffmpeg -i pop.ShawnMendesCamilaCabello.Senorita.mp4 pop.ShawnMendesCamilaCabello.Senorita.mp3"
command2wav = "ffmpeg -i pop.ShawnMendesCamilaCabello.Senorita.mp3 pop.ShawnMendesCamilaCabello.Senorita.wav"
os.system(command2mp3)
os.system("rm pop.ShawnMendesCamilaCabello.Senorita.mp4")
os.system(command2wav)
os.system("rm pop.ShawnMendesCamilaCabello.Senorita.mp3")


In [ ]:
## converting from mp4 to wav reggae
import os

command2mp3 = "ffmpeg -i reggae.BobMarley.Zimbabwe.mp4 reggae.BobMarley.Zimbabwe.mp3"
command2wav = "ffmpeg -i reggae.BobMarley.Zimbabwe.mp3 reggae.BobMarley.Zimbabwe.wav"
os.system(command2mp3)
os.system("rm reggae.BobMarley.Zimbabwe.mp4")
os.system(command2wav)
os.system("rm reggae.BobMarley.Zimbabwe.mp3")

command2mp3 = "ffmpeg -i reggae.BujuBanto.Coconutwata.mp4 reggae.BujuBanto.Coconutwata.mp3"
command2wav = "ffmpeg -i reggae.BujuBanto.Coconutwata.mp3 reggae.BujuBanto.Coconutwata.wav"
os.system(command2mp3)
os.system("rm reggae.BujuBanto.Coconutwata.mp4")
os.system(command2wav)
os.system("rm reggae.BujuBanto.Coconutwata.mp3")

command2mp3 = "ffmpeg -i reggae.Culture.Whyamirastaman.mp4 reggae.Culture.Whyamirastaman.mp3"
command2wav = "ffmpeg -i reggae.Culture.Whyamirastaman.mp3 reggae.Culture.Whyamirastaman.wav"
os.system(command2mp3)
os.system("rm reggae.Culture.Whyamirastaman.mp4")
os.system(command2wav)
os.system("rm reggae.Culture.Whyamirastaman.mp3")

command2mp3 = "ffmpeg -i reggae.Gregory.Isaacs.NightNurse.mp4 reggae.Gregory.Isaacs.NightNurse.mp3"
command2wav = "ffmpeg -i reggae.Gregory.Isaacs.NightNurse.mp3 reggae.Gregory.Isaacs.NightNurse.wav"
os.system(command2mp3)
os.system("rm reggae.Gregory.Isaacs.NightNurse.mp4")
os.system(command2wav)
os.system("rm reggae.Gregory.Isaacs.NightNurse.mp3")

command2mp3 = "ffmpeg -i reggae.PeterTosh.BushDoctor.mp4 reggae.PeterTosh.BushDoctor.mp3"
command2wav = "ffmpeg -i reggae.PeterTosh.BushDoctor.mp3 reggae.PeterTosh.BushDoctor.wav"
os.system(command2mp3)
os.system("rm reggae.PeterTosh.BushDoctor.mp4")
os.system(command2wav)
os.system("rm reggae.PeterTosh.BushDoctor.mp3")

command2mp3 = "ffmpeg -i reggae.PeterTosh.SteppingRazor.mp4 reggae.PeterTosh.SteppingRazor.mp3"
command2wav = "ffmpeg -i reggae.PeterTosh.SteppingRazor.mp3 reggae.PeterTosh.SteppingRazor.wav"
os.system(command2mp3)
os.system("rm reggae.PeterTosh.SteppingRazor.mp4")
os.system(command2wav)
os.system("rm reggae.PeterTosh.SteppingRazor.mp3")

command2mp3 = "ffmpeg -i reggae.Raggalax.Climbingtothetop.mp4 reggae.Raggalax.Climbingtothetop.mp3"
command2wav = "ffmpeg -i reggae.Raggalax.Climbingtothetop.mp3 reggae.Raggalax.Climbingtothetop.wav"
os.system(command2mp3)
os.system("rm reggae.Raggalax.Climbingtothetop.mp4")
os.system(command2wav)
os.system("rm reggae.Raggalax.Climbingtothetop.mp3")

command2mp3 = "ffmpeg -i reggae.StickFigure.EdgeoftheOcean.mp4 reggae.StickFigure.EdgeoftheOcean.mp3"
command2wav = "ffmpeg -i reggae.StickFigure.EdgeoftheOcean.mp3 reggae.StickFigure.EdgeoftheOcean.wav"
os.system(command2mp3)
os.system("rm reggae.StickFigure.EdgeoftheOcean.mp4")
os.system(command2wav)
os.system("rm reggae.StickFigure.EdgeoftheOcean.mp3")

command2mp3 = "ffmpeg -i reggae.Sublime.PawnShop.mp4 reggae.Sublime.PawnShop.mp3"
command2wav = "ffmpeg -i reggae.Sublime.PawnShop.mp3 reggae.Sublime.PawnShop.wav"
os.system(command2mp3)
os.system("rm reggae.Sublime.PawnShop.mp4")
os.system(command2wav)
os.system("rm reggae.Sublime.PawnShop.mp3")

command2mp3 = "ffmpeg -i reggae.Topgun.Somebodytolove.mp4 reggae.Topgun.Somebodytolove.mp3"
command2wav = "ffmpeg -i reggae.Topgun.Somebodytolove.mp3 reggae.Topgun.Somebodytolove.wav"
os.system(command2mp3)
os.system("rm reggae.Topgun.Somebodytolove.mp4")
os.system(command2wav)
os.system("rm reggae.Topgun.Somebodytolove.mp3")

command2mp3 = "ffmpeg -i reggae.Yellowman.Sensemilla.mp4 reggae.Yellowman.Sensemilla.mp3"
command2wav = "ffmpeg -i reggae.Yellowman.Sensemilla.mp3 reggae.Yellowman.Sensemilla.wav"
os.system(command2mp3)
os.system("rm reggae.Yellowman.Sensemilla.mp4")
os.system(command2wav)
os.system("rm reggae.Yellowman.Sensemilla.mp3")



In [ ]:
## converting from mp4 to wav rock
import os

command2mp3 = "ffmpeg -i rock.ACDC.Thunderstruck.mp4 rock.ACDC.Thunderstruck.mp3"
command2wav = "ffmpeg -i rock.ACDC.Thunderstruck.mp3 rock.ACDC.Thunderstruck.wav"
os.system(command2mp3)
os.system("rm rock.ACDC.Thunderstruck.mp4")
os.system(command2wav)
os.system("rm rock.ACDC.Thunderstruck.mp3")

command2mp3 = "ffmpeg -i rock.ArticMonkeys.DoIwannaknow.mp4 rock.ArticMonkeys.DoIwannaknow.mp3"
command2wav = "ffmpeg -i rock.ArticMonkeys.DoIwannaknow.mp3 rock.ArticMonkeys.DoIwannaknow.wav"
os.system(command2mp3)
os.system("rm rock.ArticMonkeys.DoIwannaknow.mp4")
os.system(command2wav)
os.system("rm rock.ArticMonkeys.DoIwannaknow.mp3")

command2mp3 = "ffmpeg -i rock.BonJovi.ItsmyLife.mp4 rock.BonJovi.ItsmyLife.mp3"
command2wav = "ffmpeg -i rock.BonJovi.ItsmyLife.mp3 rock.BonJovi.ItsmyLife.wav"
os.system(command2mp3)
os.system("rm rock.BonJovi.ItsmyLife.mp4")
os.system(command2wav)
os.system("rm rock.BonJovi.ItsmyLife.mp3")

command2mp3 = "ffmpeg -i rock.BreakingBenjamin.Breath.mp4 rock.BreakingBenjamin.Breath.mp3"
command2wav = "ffmpeg -i rock.BreakingBenjamin.Breath.mp3 rock.BreakingBenjamin.Breath.wav"
os.system(command2mp3)
os.system("rm rock.BreakingBenjamin.Breath.mp4")
os.system(command2wav)
os.system("rm rock.BreakingBenjamin.Breath.mp3")

command2mp3 = "ffmpeg -i rock.GreenDay.BoulevardofBrokenDreams.mp4 rock.GreenDay.BoulevardofBrokenDreams.mp3"
command2wav = "ffmpeg -i rock.GreenDay.BoulevardofBrokenDreams.mp3 rock.GreenDay.BoulevardofBrokenDreams.wav"
os.system(command2mp3)
os.system("rm rock.GreenDay.BoulevardofBrokenDreams.mp4")
os.system(command2wav)
os.system("rm rock.GreenDay.BoulevardofBrokenDreams.mp3")

command2mp3 = "ffmpeg -i rock.GunsNRoses.NovemberRain.mp4 rock.GunsNRoses.NovemberRain.mp3"
command2wav = "ffmpeg -i rock.GunsNRoses.NovemberRain.mp3 rock.GunsNRoses.NovemberRain.wav"
os.system(command2mp3)
os.system("rm rock.GunsNRoses.NovemberRain.mp4")
os.system(command2wav)
os.system("rm rock.GunsNRoses.NovemberRain.mp3")

command2mp3 = "ffmpeg -i rock.LinkinPark.Numb.mp4 rock.LinkinPark.Numb.mp3"
command2wav = "ffmpeg -i rock.LinkinPark.Numb.mp3 rock.LinkinPark.Numb.wav"
os.system(command2mp3)
os.system("rm rock.LinkinPark.Numb.mp4")
os.system(command2wav)
os.system("rm rock.LinkinPark.Numb.mp3")

command2mp3 = "ffmpeg -i rock.Queen.BohemianRhapsody.mp4 rock.Queen.BohemianRhapsody.mp3"
command2wav = "ffmpeg -i rock.Queen.BohemianRhapsody.mp3 rock.Queen.BohemianRhapsody.wav"
os.system(command2mp3)
os.system("rm rock.Queen.BohemianRhapsody.mp4")
os.system(command2wav)
os.system("rm rock.Queen.BohemianRhapsody.mp3")

command2mp3 = "ffmpeg -i rock.Radiohead.Creep.mp4 rock.Radiohead.Creep.mp3"
command2wav = "ffmpeg -i rock.Radiohead.Creep.mp3 rock.Radiohead.Creep.wav"
os.system(command2mp3)
os.system("rm rock.Radiohead.Creep.mp4")
os.system(command2wav)
os.system("rm rock.Radiohead.Creep.mp3")

command2mp3 = "ffmpeg -i rock.RedHotChillipeppers.Californication.mp4 rock.RedHotChillipeppers.Californication.mp3"
command2wav = "ffmpeg -i rock.RedHotChillipeppers.Californication.mp3 rock.RedHotChillipeppers.Californication.wav"
os.system(command2mp3)
os.system("rm rock.RedHotChillipeppers.Californication.mp4")
os.system(command2wav)
os.system("rm rock.RedHotChillipeppers.Californication.mp3")

command2mp3 = "ffmpeg -i rock.TheCranberries.Zombie.mp4 rock.TheCranberries.Zombie.mp3"
command2wav = "ffmpeg -i rock.TheCranberries.Zombie.mp3 rock.TheCranberries.Zombie.wav"
os.system(command2mp3)
os.system("rm rock.TheCranberries.Zombie.mp4")
os.system(command2wav)
os.system("rm rock.TheCranberries.Zombie.mp3")



In [ ]:
## Updates dicts with new pathfiles from mp4 to wav
classical_dict = {'file_name': ['classical.Tchaikovsky.PianoN1',
                                'classical.Mozart.LNDF',
                                'classical.Greig.MountKing',
                                'classical.Elgar.MarchPomp',
                                'classical.Schubert.MMN1', 
                                'classical.Rossini.BarberSevilla', 
                                'classical.Puccini.NessumDorma',
                                'classical.Beethoven.PianokonsertN5Bach.TOFStravinsky.Petrushka',
                                'classical.HandelMessiah.hallelujah',
                                'classical.FrederocChopin.FuneralMarch',
                                'classical.Bach.ToccataNFugue'],
                  'file_path' : ['classical.Tchaikovsky.PianoN1.wav',
                                 'classical.Mozart.LNDF.wav',
                                 'classical.Greig.MountKing.wav',
                                 'classical.Elgar.MarchPomp.wav',
                                 'classical.Schubert.MMN1.wav', 
                                 'classical.Rossini.BarberSevilla.wav', 
                                 'classical.Puccini.NessumDorma.wav',
                                 'classical.Beethoven.PianokonsertN5Bach.TOFStravinsky.Petrushka.wav',
                                  'classical.HandelMessiah.hallelujah.wav',
                                 'classical.FrederocChopin.FuneralMarch.wav',
                                 'classical.Bach.ToccataNFugue.wav']
                  }

blues_dict = {'file_name': ['blues.WrecklessStrangers.FastGirls',
                            'blues.TheBarrBrothers.LordIJustCant',
                            'blues.TheBarrBrothers.HalfCrazy',
                            'blues.StevieRayVaughan.LittleWing',
                            'blues.RichardThompson.IfLoveWhispersYour', 
                            'blues.GaryColeman.TheSkyisCrying', 
                            'blues.DavidDeacon.TheyAllSangAlong',
                            'blues.DanielCastro.IllPlayTheBluesForYou',
                            'blues.BuddyGuy.FleshNBone',
                            'blues.AnnieLennoxIPutASpellOnYou',
                            'blues.Andra.AtLast'],
                  'file_path' : ['blues.WrecklessStrangers.FastGirls.wav',
                                 'blues.TheBarrBrothers.LordIJustCant.wav',
                                 'blues.TheBarrBrothers.HalfCrazy.wav',
                                 'blues.StevieRayVaughan.LittleWing.wav',
                                 'blues.RichardThompson.IfLoveWhispersYour.wav', 
                                 'blues.GaryColeman.TheSkyisCrying.wav', 
                                 'blues.DavidDeacon.TheyAllSangAlong.wav',
                                 'blues.DanielCastro.IllPlayTheBluesForYou.wav',
                                 'blues.BuddyGuy.FleshNBone.wav',
                                 'blues.AnnieLennoxIPutASpellOnYou.wav',
                                 'blues.Andra.AtLast.wav']
                  }

country_dict = {'file_name': ['country.AlanJackson.RememberWhen',
                              'country.BillliCurrington.PeopleareCrazy',
                              'country.ChaseMatthew.WeAllGrowUp',
                              'country.CodyJohnson.Thepainter',
                              'country.DariusRucker.WagonWheel', 
                              'country.KennyChesney.TakeHerHome', 
                              'country.LadyAntebellum.NeedYouNow',
                              'country.LeeBrice.OneofThemGirls',
                              'country.LukeCombs.BeautifulCrazy',
                              'country.Midlan.DrinkinProblem',
                              'country.ShaniaTwain.Ifeellikeawoman'],
                  'file_path' : ['country.AlanJackson.RememberWhen.wav',
                                 'country.BillliCurrington.PeopleareCrazy.wav',
                                 'country.ChaseMatthew.WeAllGrowUp.wav',
                                 'country.CodyJohnson.Thepainter.wav',
                                 'country.DariusRucker.WagonWheel.wav', 
                                 'country.KennyChesney.TakeHerHome.wav', 
                                 'country.LadyAntebellum.NeedYouNow.wav',
                                 'country.LeeBrice.OneofThemGirls.wav',
                                 'country.LukeCombs.BeautifulCrazy.wav',
                                 'country.Midlan.DrinkinProblem.wav',
                                 'country.ShaniaTwain.Ifeellikeawoman.wav']
                  }

disco_dict = {'file_name': ['disco.VillagePeople.YMCA',
                            'disco.VanMcCoy.Thehustle',
                            'disco.Sylvester.Doyouwantfunk',
                            'disco.RickJames.SuperFreak',
                            'disco.MichaelJackson.BillieJean', 
                            'disco.KoolNTheGang.Celebration', 
                            'disco.JackieMoore.Thistimebaby',
                            'disco.GloriaGaynor.Iwillsurvive',
                            'disco.FranceJoli.Cometome',
                            'disco.DonnaSummer.Lovetoloveyoubaby',
                            'disco.BeeGees.NightFever'],
                  'file_path' : ['disco.VillagePeople.YMCA.wav',
                                 'disco.VanMcCoy.Thehustle.wav',
                                 'disco.Sylvester.Doyouwantfunk.wav',
                                 'disco.RickJames.SuperFreak.wav',
                                 'disco.MichaelJackson.BillieJean.wav', 
                                 'disco.KoolNTheGang.Celebration.wav', 
                                 'disco.JackieMoore.Thistimebaby.wav',
                                 'disco.GloriaGaynor.Iwillsurvive.wav',
                                 'disco.FranceJoli.Cometome.wav',
                                 'disco.DonnaSummer.Lovetoloveyoubaby.wav',
                                 'disco.BeeGees.NightFever.wav']
                  }

hiphop_dict = {'file_name': ['hiphop.Wu-TangClan.ProtectYaNeck',
                             'hiphop.TheNotorious.Juicy',
                             'hiphop.SnoopDogg.GinandJuice',
                             'hiphop.Nas.NYStateofMind',
                             'hiphop.MobbDeep.ShookOnes',
                             'hiphop.Future.KeepItBurnin', 
                             'hiphop.EricBRakim.PaidinFull',
                             'hiphop.Eminem.LoseYourself',
                             'hiphop.DrDre.NuthinbutaGthang',
                             'hiphop.Coolio.GangstaParadise',
                             'hiphop.2Pac.HitEmUp'],
                  'file_path' : ['hiphop.Wu-TangClan.ProtectYaNeck.wav',
                                 'hiphop.TheNotorious.Juicy.wav',
                                 'hiphop.SnoopDogg.GinandJuice.wav',
                                 'hiphop.Nas.NYStateofMind.wav',
                                 'hiphop.MobbDeep.ShookOnes.wav', 
                                 'hiphop.Future.KeepItBurnin.wav',
                                 'hiphop.EricBRakim.PaidinFull.wav',
                                 'hiphop.Eminem.LoseYourself.wav',
                                 'hiphop.DrDre.NuthinbutaGthang.wav',
                                 'hiphop.Coolio.GangstaParadise.wav',
                                 'hiphop.2Pac.HitEmUp.wav']
                  }

jazz_dict = {'file_name': ['jazz.OliverNelson.StolenMoments',
                           'jazz.MilesDavis.IfIWereABell',
                           'jazz.MilesDavis.FreddieFreeloader',
                           'jazz.JohnColtrane.Naima',
                           'jazz.JimmySmith.WhenJohnnycomesmarching', 
                           'jazz.GrooveHolmes.Misty', 
                           'jazz.FreddieHubbard.LittleSunflow',
                           'jazz.DaveBrubeck.TakeFive',
                           'jazz.CharlieBirbParker.YardbirdSuite',
                           'jazz.ArtPepper.Youbesoniceto',
                           'jazz.ArtFarmer.KillerJoe'],
                  'file_path' : ['jazz.OliverNelson.StolenMoments.wav',
                                 'jazz.MilesDavis.IfIWereABell.wav',
                                 'jazz.MilesDavis.FreddieFreeloader.wav',
                                 'jazz.JohnColtrane.Naima.wav',
                                 'jazz.JimmySmith.WhenJohnnycomesmarching.wav',
                                 'jazz.GrooveHolmes.Misty.wav', 
                                 'jazz.FreddieHubbard.LittleSunflow.wav',
                                 'jazz.DaveBrubeck.TakeFive.wav',
                                 'jazz.CharlieBirbParker.YardbirdSuite.wav',
                                 'jazz.ArtPepper.Youbesoniceto.wav',
                                 'jazz.ArtFarmer.KillerJoe.wav']
                  }

metal_dict = {'file_name': ['metal.Venom.Welcometohell',
                            'metal.Tool.Enema',
                            'metal.Slipknot.WaitNBleed',
                            'metal.Pantera.Walk',
                            'metal.Motorhead.AceofSpades', 
                            'metal.Metallica.FadetoBlack', 
                            'metal.Korn.Freakonaleash',
                            'metal.KillswitchEngage.TheSignalFire',
                            'metal.IronMaiden.Runtothehills',
                            'metal.BlackSabbath.Paranoid',
                            'metal.AliceinChains.WeDieYoung'],
                  'file_path' : ['metal.Venom.Welcometohell.wav',
                                 'metal.Tool.Enema.wav',
                                 'metal.Slipknot.WaitNBleed.wav',
                                 'metal.Pantera.Walk.wav',
                                 'metal.Motorhead.AceofSpades.wav', 
                                 'metal.Metallica.FadetoBlack.wav', 
                                 'metal.Korn.Freakonaleash.wav',
                                 'metal.KillswitchEngage.TheSignalFire.wav',
                                 'metal.IronMaiden.Runtothehills.wav',
                                 'metal.BlackSabbath.Paranoid.wav',
                                 'metal.AliceinChains.WeDieYoung.wav']
                  }

pop_dict = {'file_name': ['pop.Adele.EasyonMe',
                          'pop.BrunoMars.Thelazysong',
                          'pop.CharliePuth.WeDonttalkanymore',
                          'pop.DuaLipa.NewRules',
                          'pop.EdSheeran.ShapeofYou', 
                          'pop.JustinBieber.Ghost', 
                          'pop.Ladygaga.Pokerface',
                          'pop.LewisCapaldi.Someoneyouloved',
                          'pop.MileyCyrus.Flowers',
                          'pop.Passenger.Lethergo',
                          'pop.ShawnMendesCamilaCabello.Senorita'],
                  'file_path' : ['pop.Adele.EasyonMe.wav',
                                 'pop.BrunoMars.Thelazysong.wav',
                                 'pop.CharliePuth.WeDonttalkanymore.wav',
                                 'pop.DuaLipa.NewRules.wav',
                                 'pop.EdSheeran.ShapeofYou.wav', 
                                 'pop.JustinBieber.Ghost.wav', 
                                 'pop.Ladygaga.Pokerface.wav',
                                 'pop.LewisCapaldi.Someoneyouloved.wav',
                                 'pop.MileyCyrus.Flowers.wav',
                                 'pop.Passenger.Lethergo.wav',
                                 'pop.ShawnMendesCamilaCabello.Senorita.wav']
                  }

reggae_dict = {'file_name': ['reggae.BobMarley.Zimbabwe',
                             'reggae.BujuBanto.Coconutwata',
                             'reggae.Culture.Whyamirastaman',
                             'reggae.Gregory.Isaacs.NightNurse',
                             'reggae.PeterTosh.BushDoctor', 
                             'reggae.PeterTosh.SteppingRazor', 
                             'reggae.Raggalax.Climbingtothetop',
                             'reggae.StickFigure.EdgeoftheOcean',
                             'reggae.Sublime.PawnShop',
                             'reggae.Topgun.Somebodytolove',
                             'reggae.Yellowman.Sensemilla'],
                  'file_path' : ['reggae.BobMarley.Zimbabwe.wav',
                                 'reggae.BujuBanto.Coconutwata.wav',
                                 'reggae.Culture.Whyamirastaman.wav',
                                 'reggae.Gregory.Isaacs.NightNurse.wav',
                                 'reggae.PeterTosh.BushDoctor.wav', 
                                 'reggae.PeterTosh.SteppingRazor.wav', 
                                 'reggae.Raggalax.Climbingtothetop.wav',
                                 'reggae.StickFigure.EdgeoftheOcean.wav',
                                 'reggae.Sublime.PawnShop.wav',
                                 'reggae.Topgun.Somebodytolove.wav',
                                 'reggae.Yellowman.Sensemilla.wav']
                  }

rock_dict = {'file_name': ['rock.ACDC.Thunderstruck',
                           'rock.ArticMonkeys.DoIwannaknow',
                           'rock.BonJovi.ItsmyLife',
                           'rock.BreakingBenjamin.Breath',
                           'rock.GreenDay.BoulevardofBrokenDreams', 
                           'rock.GunsNRoses.NovemberRain', 
                           'rock.LinkinPark.Numb',
                           'rock.Queen.BohemianRhapsody',
                           'rock.Radiohead.Creep',
                           'rock.RedHotChillipeppers.Californication',
                           'rock.TheCranberries.Zombie'],
                  'file_path' : ['rock.ACDC.Thunderstruck.wav',
                                 'rock.ArticMonkeys.DoIwannaknow.wav',
                                 'rock.BonJovi.ItsmyLife.wav',
                                 'rock.BreakingBenjamin.Breath.wav',
                                 'rock.GreenDay.BoulevardofBrokenDreams.wav', 
                                 'rock.GunsNRoses.NovemberRain.wav', 
                                 'rock.LinkinPark.Numb.wav',
                                 'rock.Queen.BohemianRhapsody.wav',
                                 'rock.Radiohead.Creep.wav',
                                 'rock.RedHotChillipeppers.Californication.wav',
                                 'rock.TheCranberries.Zombie.wav']
                  }


In [ ]:
## breaking new data set into 2 sample songs of 30 seconds = 20 x 30s samples/genre
for i in range(0, len(rock_dict['file_name'])):
    # crop_file_to_30_sec(classical_dict['file_path'][i], classical_dict['file_name'][i])
    
    # crop_file_to_30_sec(blues_dict['file_path'][i], blues_dict['file_name'][i])
    
    # crop_file_to_30_sec(country_dict['file_path'][i], country_dict['file_name'][i])
    
    # crop_file_to_30_sec(disco_dict['file_path'][i], disco_dict['file_name'][i])
    
    # crop_file_to_30_sec(hiphop_dict['file_path'][i], hiphop_dict['file_name'][i])
    
    # crop_file_to_30_sec(jazz_dict['file_path'][i], jazz_dict['file_name'][i])
    
    # crop_file_to_30_sec(metal_dict['file_path'][i], metal_dict['file_name'][i])
    
    # crop_file_to_30_sec(pop_dict['file_path'][i], pop_dict['file_name'][i])
    
    # crop_file_to_30_sec(reggae_dict['file_path'][i], reggae_dict['file_name'][i])
    
    crop_file_to_30_sec(rock_dict['file_path'][i], rock_dict['file_name'][i])
    

In [ ]:
## Get songs for demo set = 2/genre
classical_dict = {'file_name': ['Mozart.DerHolleRache',
                                'Beethoven.FurElise'
                                ],
                  'file_url' : ['https://youtu.be/-mL5_f_ORHI?si=hA5Oh7N5WgrqnN3g',
                                'https://youtu.be/p7zUi0-zflM?si=crkMbaSuzC8j5P9M'
                                ]
                  }

blues_dict = {'file_name': ['MeenaCryle.ItMakesMeScream',
                            'AlvinLee.TheBluestBlues'],
                  'file_url' : ['https://youtu.be/rnQzXv-bbkY?si=YdX4yIREyNmocoW9',
                                'https://youtu.be/rB6OlJqV1rQ?si=j6x5lQQ0TeqCdkC3']
                  }
country_dict = {'file_name': ['AlanJackson.Chattahoochee',
                              'TobyKeith.AsGoodasIoncewas'],
                  'file_url' : ['https://youtu.be/JW5UEW2kYvc?si=c5JiCFzWAfoYsRuo',
                                'https://youtu.be/ldQrapQ4d0Y?si=acFf71-lQqT3H30H']
                  }
disco_dict = {'file_name': ['EarthWindFire.September',
                            'TheTrammps.ThatsWhereTheHappyPeopleGo'],
                  'file_url' : ['https://youtu.be/Gs069dndIYk?si=iRrxaGKagfPqWn3I',
                                'https://youtu.be/Tqg8l4nsFEw?si=F9gCAZVEg5PeTGa9']
                  }
hiphop_dict = {'file_name': ['DrDre.StillDRE',
                             '50cent.InDaClub'],
                  'file_url' : ['https://youtu.be/_CL6n0FJZpk?si=Md59pXxZ9-YAnI9f',
                                'https://youtu.be/5qm8PH4xAss?si=exAdf77pg8aCpI4Z']
                  }
jazz_dict = {'file_name': ['DukePearson.ChilliPeppers',
                           'HankMobbley.Remember'],
                  'file_url' : ['https://youtu.be/YjRWditGKRY?si=GS73q1RC22tbX5rx',
                                'https://youtu.be/KV0HX9sk_04?si=fiiRQOMRMS-UI78f']
                  }
metal_dict = {'file_name': ['Sysemofadown.ChopSuey',
                            'Opeth.BeneaththeMire'],
                  'file_url' : ['https://youtu.be/CSvFpBOe8eY?si=HM2bUxyl4KnNtTpg',
                                'https://youtu.be/QlT1sRHf34M?si=wnFK9gAmnIoCXuGP']
                  }
pop_dict = {'file_name': ['BillieEilish.WhatwasImadefor',
                          'Maroon5.Sugar'],
                  'file_url' : ['https://youtu.be/cW8VLC9nnTo?si=bRAsPiP9WrW44wO-',
                                'https://youtu.be/09R8_2nJtjg?si=SXlKw8w81_t8pQh4']
                  }
reggae_dict = {'file_name': ['BobMarley.Ishotthesheriff',
                             'Soja.JahAtmosphere'],
                  'file_url' : ['https://youtu.be/qsYzKtnvHfs?si=oNSK-rZSUSn9ce5L',
                                'https://youtu.be/94HwzFGH5gY?si=zndza0LBYX7YnChg']
                  }
rock_dict = {'file_name': ['FooFighters.Thepretender',
                           'Starset.Mydemons'],
                  'file_url' : ['https://youtu.be/SBjQ9tuuTJQ?si=gyKboN_yJdkJtOqo',
                                'https://youtu.be/p-N_y1bZtRw?si=DUV2U4wgE3mzOYot']
                  }



In [ ]:
## downloading files in dicts 
for i in range(0, len(classical_dict['file_name'])):
    get_youtube_dl_audio_file('classical', classical_dict['file_url'][i], classical_dict['file_name'][i])
    
    get_youtube_dl_audio_file('blues', blues_dict['file_url'][i], blues_dict['file_name'][i])
    
    get_youtube_dl_audio_file('country', country_dict['file_url'][i], country_dict['file_name'][i])
    
    get_youtube_dl_audio_file('disco', disco_dict['file_url'][i], disco_dict['file_name'][i])
    
    get_youtube_dl_audio_file('hiphop', hiphop_dict['file_url'][i], hiphop_dict['file_name'][i])
    
    get_youtube_dl_audio_file('jazz', jazz_dict['file_url'][i], jazz_dict['file_name'][i])
    
    get_youtube_dl_audio_file('metal', metal_dict['file_url'][i], metal_dict['file_name'][i])
    
    get_youtube_dl_audio_file('pop', pop_dict['file_url'][i], pop_dict['file_name'][i])
    
    get_youtube_dl_audio_file('reggae', reggae_dict['file_url'][i], reggae_dict['file_name'][i])
    
    get_youtube_dl_audio_file('rock', rock_dict['file_url'][i], rock_dict['file_name'][i])
    

In [ ]:
## converting from mp4 to wav rock
import os

command2mp3 = "ffmpeg -i rock.FooFighters.Thepretender.mp4 rock.FooFighters.Thepretender.mp3"
command2wav = "ffmpeg -i rock.FooFighters.Thepretender.mp3 rock.FooFighters.Thepretender.wav"
os.system(command2mp3)
os.system("rm rock.FooFighters.Thepretender.mp4")
os.system(command2wav)
os.system("rm rock.FooFighters.Thepretender.mp3")

command2mp3 = "ffmpeg -i rock.Starset.Mydemons.mp4 rock.Starset.Mydemons.mp3"
command2wav = "ffmpeg -i rock.Starset.Mydemons.mp3 rock.Starset.Mydemons.wav"
os.system(command2mp3)
os.system("rm rock.Starset.Mydemons.mp4")
os.system(command2wav)
os.system("rm rock.Starset.Mydemons.mp3")

command2mp3 = "ffmpeg -i reggae.BobMarley.Ishotthesheriff.mp4 reggae.BobMarley.Ishotthesheriff.mp3"
command2wav = "ffmpeg -i reggae.BobMarley.Ishotthesheriff.mp3 reggae.BobMarley.Ishotthesheriff.wav"
os.system(command2mp3)
os.system("rm reggae.BobMarley.Ishotthesheriff.mp4")
os.system(command2wav)
os.system("rm reggae.BobMarley.Ishotthesheriff.mp3")

command2mp3 = "ffmpeg -i reggae.Soja.JahAtmosphere.mp4 reggae.Soja.JahAtmosphere.mp3"
command2wav = "ffmpeg -i reggae.Soja.JahAtmosphere.mp3 reggae.Soja.JahAtmosphere.wav"
os.system(command2mp3)
os.system("rm reggae.Soja.JahAtmosphere.mp4")
os.system(command2wav)
os.system("rm reggae.Soja.JahAtmosphere.mp3")

command2mp3 = "ffmpeg -i pop.BillieEilish.WhatwasImadefor.mp4 pop.BillieEilish.WhatwasImadefor.mp3"
command2wav = "ffmpeg -i pop.BillieEilish.WhatwasImadefor.mp3 pop.BillieEilish.WhatwasImadefor.wav"
os.system(command2mp3)
os.system("rm pop.BillieEilish.WhatwasImadefor.mp4")
os.system(command2wav)
os.system("rm pop.BillieEilish.WhatwasImadefor.mp3")

command2mp3 = "ffmpeg -i pop.Maroon5.Sugar.mp4 pop.Maroon5.Sugar.mp3"
command2wav = "ffmpeg -i pop.Maroon5.Sugar.mp3 pop.Maroon5.Sugar.wav"
os.system(command2mp3)
os.system("rm pop.Maroon5.Sugar.mp4")
os.system(command2wav)
os.system("rm pop.Maroon5.Sugar.mp3")

command2mp3 = "ffmpeg -i hiphop.50cent.InDaClub.mp4 hiphop.50cent.InDaClub.mp3"
command2wav = "ffmpeg -i hiphop.50cent.InDaClub.mp3 hiphop.50cent.InDaClub.wav"
os.system(command2mp3)
os.system("rm hiphop.50cent.InDaClub.mp4")
os.system(command2wav)
os.system("rm hiphop.50cent.InDaClub.mp3")

command2mp3 = "ffmpeg -i hiphop.DrDre.StillDRE.mp4 hiphop.DrDre.StillDRE.mp3"
command2wav = "ffmpeg -i hiphop.DrDre.StillDRE.mp3 hiphop.DrDre.StillDRE.wav"
os.system(command2mp3)
os.system("rm hiphop.DrDre.StillDRE.mp4")
os.system(command2wav)
os.system("rm hiphop.DrDre.StillDRE.mp3")

command2mp3 = "ffmpeg -i metal.Opeth.BeneaththeMire.mp4 metal.Opeth.BeneaththeMire.mp3"
command2wav = "ffmpeg -i metal.Opeth.BeneaththeMire.mp3 metal.Opeth.BeneaththeMire.wav"
os.system(command2mp3)
os.system("rm metal.Opeth.BeneaththeMire.mp4")
os.system(command2wav)
os.system("rm metal.Opeth.BeneaththeMire.mp3")

command2mp3 = "ffmpeg -i metal.Sysemofadown.ChopSuey.mp4 metal.Sysemofadown.ChopSuey.mp3"
command2wav = "ffmpeg -i metal.Sysemofadown.ChopSuey.mp3 metal.Sysemofadown.ChopSuey.wav"
os.system(command2mp3)
os.system("rm metal.Sysemofadown.ChopSuey.mp4")
os.system(command2wav)
os.system("rm metal.Sysemofadown.ChopSuey.mp3")

command2mp3 = "ffmpeg -i jazz.DukePearson.ChilliPeppers.mp4 jazz.DukePearson.ChilliPeppers.mp3"
command2wav = "ffmpeg -i jazz.DukePearson.ChilliPeppers.mp3 jazz.DukePearson.ChilliPeppers.wav"
os.system(command2mp3)
os.system("rm jazz.DukePearson.ChilliPeppers.mp4")
os.system(command2wav)
os.system("rm jazz.DukePearson.ChilliPeppers.mp3")

command2mp3 = "ffmpeg -i jazz.HankMobbley.Remember.mp4 jazz.HankMobbley.Remember.mp3"
command2wav = "ffmpeg -i jazz.HankMobbley.Remember.mp3 jazz.HankMobbley.Remember.wav"
os.system(command2mp3)
os.system("rm jazz.HankMobbley.Remember.mp4")
os.system(command2wav)
os.system("rm jazz.HankMobbley.Remember.mp3")

command2mp3 = "ffmpeg -i disco.EarthWindFire.September.mp4 disco.EarthWindFire.September.mp3"
command2wav = "ffmpeg -i disco.EarthWindFire.September.mp3 disco.EarthWindFire.September.wav"
os.system(command2mp3)
os.system("rm disco.EarthWindFire.September.mp4")
os.system(command2wav)
os.system("rm disco.EarthWindFire.September.mp3")

command2mp3 = "ffmpeg -i disco.TheTrammps.ThatsWhereTheHappyPeopleGo.mp4 disco.TheTrammps.ThatsWhereTheHappyPeopleGo.mp3"
command2wav = "ffmpeg -i disco.TheTrammps.ThatsWhereTheHappyPeopleGo.mp3 disco.TheTrammps.ThatsWhereTheHappyPeopleGo.wav"
os.system(command2mp3)
os.system("rm disco.TheTrammps.ThatsWhereTheHappyPeopleGo.mp4")
os.system(command2wav)
os.system("rm disco.TheTrammps.ThatsWhereTheHappyPeopleGo.mp3")

command2mp3 = "ffmpeg -i country.AlanJackson.Chattahoochee.mp4 country.AlanJackson.Chattahoochee.mp3"
command2wav = "ffmpeg -i country.AlanJackson.Chattahoochee.mp3 country.AlanJackson.Chattahoochee.wav"
os.system(command2mp3)
os.system("rm country.AlanJackson.Chattahoochee.mp4")
os.system(command2wav)
os.system("rm country.AlanJackson.Chattahoochee.mp3")

command2mp3 = "ffmpeg -i country.TobyKeith.AsGoodasIoncewas.mp4 country.TobyKeith.AsGoodasIoncewas.mp3"
command2wav = "ffmpeg -i country.TobyKeith.AsGoodasIoncewas.mp3 country.TobyKeith.AsGoodasIoncewas.wav"
os.system(command2mp3)
os.system("rm country.TobyKeith.AsGoodasIoncewas.mp4")
os.system(command2wav)
os.system("rm country.TobyKeith.AsGoodasIoncewas.mp3")

command2mp3 = "ffmpeg -i blues.AlvinLee.TheBluestBlues.mp4 blues.AlvinLee.TheBluestBlues.mp3"
command2wav = "ffmpeg -i blues.AlvinLee.TheBluestBlues.mp3 blues.AlvinLee.TheBluestBlues.wav"
os.system(command2mp3)
os.system("rm blues.AlvinLee.TheBluestBlues.mp4")
os.system(command2wav)
os.system("rm blues.AlvinLee.TheBluestBlues.mp3")

command2mp3 = "ffmpeg -i blues.MeenaCryle.ItMakesMeScream.mp4 blues.MeenaCryle.ItMakesMeScream.mp3"
command2wav = "ffmpeg -i blues.MeenaCryle.ItMakesMeScream.mp3 blues.MeenaCryle.ItMakesMeScream.wav"
os.system(command2mp3)
os.system("rm blues.MeenaCryle.ItMakesMeScream.mp4")
os.system(command2wav)
os.system("rm blues.MeenaCryle.ItMakesMeScream.mp3")

command2mp3 = "ffmpeg -i classical.Beethoven.FurElise.mp4 classical.Beethoven.FurElise.mp3"
command2wav = "ffmpeg -i classical.Beethoven.FurElise.mp3 classical.Beethoven.FurElise.wav"
os.system(command2mp3)
os.system("rm classical.Beethoven.FurElise.mp4")
os.system(command2wav)
os.system("rm classical.Beethoven.FurElise.mp3")

command2mp3 = "ffmpeg -i classical.Mozart.DerHolleRache.mp4 classical.Mozart.DerHolleRache.mp3"
command2wav = "ffmpeg -i classical.Mozart.DerHolleRache.mp3 classical.Mozart.DerHolleRache.wav"
os.system(command2mp3)
os.system("rm classical.Mozart.DerHolleRache.mp4")
os.system(command2wav)
os.system("rm classical.Mozart.DerHolleRache.mp3")

In [ ]:
## Update dicts with new wav file names and add path
classical_dict = {'file_name': ['classical.Beethoven.FurElise',
                                'classical.Mozart.DerHolleRache'],
                  'file_path' : ['classical.Beethoven.FurElise.wav',
                                'classical.Mozart.DerHolleRache.wav']
                  }
blues_dict = {'file_name': ['blues.AlvinLee.TheBluestBlues',
                            'blues.MeenaCryle.ItMakesMeScream'],
                  'file_path' : ['blues.AlvinLee.TheBluestBlues.wav',
                            'blues.MeenaCryle.ItMakesMeScream.wav']
                  }
country_dict = {'file_name': ['country.AlanJackson.Chattahoochee',
                              'country.TobyKeith.AsGoodasIoncewas'],
                  'file_path' : ['country.AlanJackson.Chattahoochee.wav',
                              'country.TobyKeith.AsGoodasIoncewas.wav']
                  }
disco_dict = {'file_name': ['disco.EarthWindFire.September',
                            'disco.TheTrammps.ThatsWhereTheHappyPeopleGo'],
                  'file_path' : ['disco.EarthWindFire.September.wav',
                            'disco.TheTrammps.ThatsWhereTheHappyPeopleGo.wav']
                  }
hiphop_dict = {'file_name': ['hiphop.50cent.InDaClub',
                             'hiphop.DrDre.StillDRE'],
                  'file_path' : ['hiphop.50cent.InDaClub.wav',
                             'hiphop.DrDre.StillDRE.wav']
                  }
jazz_dict = {'file_name': ['jazz.DukePearson.ChilliPeppers',
                           'jazz.HankMobbley.Remember'],
                  'file_path' : ['jazz.DukePearson.ChilliPeppers.wav',
                           'jazz.HankMobbley.Remember.wav']
                  }
metal_dict = {'file_name': ['metal.Opeth.BeneaththeMire',
                            'metal.Sysemofadown.ChopSuey'],
                  'file_path' : ['metal.Opeth.BeneaththeMire.wav',
                            'metal.Sysemofadown.ChopSuey.wav']
                  }
pop_dict = {'file_name': ['pop.BillieEilish.WhatwasImadefor',
                          'pop.Maroon5.Sugar'],
                  'file_path' : ['pop.BillieEilish.WhatwasImadefor.wav',
                          'pop.Maroon5.Sugar.wav']
                  }
reggae_dict = {'file_name': ['reggae.BobMarley.Ishotthesheriff',
                             'reggae.Soja.JahAtmosphere'],
                  'file_path' : ['reggae.BobMarley.Ishotthesheriff.wav',
                             'reggae.Soja.JahAtmosphere.wav']
                  }
rock_dict = {'file_name': ['rock.FooFighters.Thepretender',
                           'rock.Starset.Mydemons'],
                  'file_path' : ['rock.FooFighters.Thepretender.wav',
                           'rock.Starset.Mydemons.wav']
                  }

In [ ]:
## breaking new data set into 2 sample songs of 30 seconds = 20 x 30s samples/genre
for i in range(0, len(rock_dict['file_name'])):
    crop_file_to_30_sec(classical_dict['file_path'][i], classical_dict['file_name'][i])
    
    crop_file_to_30_sec(blues_dict['file_path'][i], blues_dict['file_name'][i])
    
    crop_file_to_30_sec(country_dict['file_path'][i], country_dict['file_name'][i])
    
    crop_file_to_30_sec(disco_dict['file_path'][i], disco_dict['file_name'][i])
    
    crop_file_to_30_sec(hiphop_dict['file_path'][i], hiphop_dict['file_name'][i])
    
    crop_file_to_30_sec(jazz_dict['file_path'][i], jazz_dict['file_name'][i])
    
    crop_file_to_30_sec(metal_dict['file_path'][i], metal_dict['file_name'][i])
    
    crop_file_to_30_sec(pop_dict['file_path'][i], pop_dict['file_name'][i])
    
    crop_file_to_30_sec(reggae_dict['file_path'][i], reggae_dict['file_name'][i])
    
    crop_file_to_30_sec(rock_dict['file_path'][i], rock_dict['file_name'][i])
    